In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install SQLAlchemy psycopg2-binary

import pandas as pd
import numpy as np
import psycopg2

from sqlalchemy import create_engine

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 13.5 MB/s eta 0:00:00


In [ ]:
engine = create_engine(CONN_STR, pool_pre_ping=True)

In [ ]:
with engine.begin() as conn:
    conn.exec_driver_sql("SET search_path TO ecommerce;")  # 접두어 기본값 설정


✅ PostgreSQL 연결 성공!  테이블 총 13 개



,table_schema,table_name
0,ecommerce,coupon
1,ecommerce,customer_info
2,ecommerce,customers
3,ecommerce,events
4,ecommerce,marketing_campaign
5,ecommerce,order_items
6,ecommerce,order_payments
7,ecommerce,order_reviews
8,ecommerce,orders
9,ecommerce,product_categories


In [ ]:
#rm -f output/predict.jsonl output/pred_dev.sql logs/gemini_run.log

In [ ]:
df.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,gender,age_group
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,female,30s
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,male,40s
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,male,40s
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,male,40s
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,female,30s


In [ ]:
!pip install -q langgraph


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 783.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.5/152.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 11.0 MB/s eta 0:00:00


In [ ]:
import os

# ▶ 반드시 본인 키로 교체
os.environ["GOOGLE_API_KEY"] = ""

# 사용 모델 (지원 안 되면 'gemini-1.5-pro'gemini-2.5-flash로 바꿔보세요)
GEMINI_MODEL = "gemini-2.5-flash"


In [ ]:
from sqlalchemy import inspect
# 디버그 출력
DEBUG = True
# =============================================================

from __future__ import annotations

import re
import json
import dataclasses
from dataclasses import dataclass, field
from typing import List, Optional

import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.engine import Engine
import networkx as nx           # ⭐ NEW: FK 그래프
import sqlglot
from langgraph.graph import StateGraph, END

# ---------------- Gemini client (google-genai) ----------------
from google import genai
from google.genai import types as genai_types

class GeminiClient:
    """google-genai SDK 래퍼. 'system' 역할 없이 본문에 병합하여 role='user'로 전달."""
    def __init__(self, model: str):
        self.client = genai.Client()
        self.model = model

    def generate(self, prompt: str, system_instruction: Optional[str] = None, json_mode: bool = False) -> str:
        cfg = genai_types.GenerateContentConfig(
            temperature=0.1,  # 결정적
            response_mime_type="application/json" if json_mode else None,
            max_output_tokens=8192,  # 길게
        )
        full_prompt = (system_instruction + "\n\n" + prompt) if system_instruction else prompt
        contents = [
            genai_types.Content(role="user", parts=[genai_types.Part(text=full_prompt)])
        ]
        resp = self.client.models.generate_content(
            model=self.model, contents=contents, config=cfg
        )
        return resp.text

# ---------------- DB utilities ----------------
def make_engine(conn_str: str) -> Engine:
    return create_engine(conn_str)

def schema_exists(engine: Engine, schema: str) -> bool:
    q = text("""
        SELECT EXISTS (
            SELECT 1 FROM information_schema.schemata WHERE schema_name = :schema
        ) AS ok
    """)
    with engine.connect() as conn:
        return pd.read_sql(q, conn, params={"schema": schema}).iloc[0,0]

def list_tables(engine: Engine, schema: str) -> pd.DataFrame:
    q = text("""
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = :schema
        ORDER BY table_name
    """)
    with engine.connect() as conn:
        return pd.read_sql(q, conn, params={"schema": schema})

def fetch_columns(engine: Engine, schema: str) -> pd.DataFrame:
    q = text("""
        SELECT table_name, column_name, data_type, ordinal_position
        FROM information_schema.columns
        WHERE table_schema = :schema
        ORDER BY table_name, ordinal_position
    """)
    with engine.connect() as conn:
        return pd.read_sql(q, conn, params={"schema": schema})

# ⭐ NEW: 스키마 간 FK 관계 그래프를 추출해 Selector 토큰 절약 + join 힌트 제공

def build_fk_graph(engine: Engine, schema: str) -> nx.DiGraph:
    insp = inspect(engine)
    g = nx.DiGraph()
    for tbl in insp.get_table_names(schema=schema):
        g.add_node(tbl)
        for fk in insp.get_foreign_keys(tbl, schema=schema):
            parent = fk["referred_table"]
            g.add_edge(parent, tbl, weight=1)
    return g

def graph_summary_text(g: nx.DiGraph) -> str:
    pr = nx.pagerank(g)
    hubs = sorted(pr.items(), key=lambda x: x[1], reverse=True)[:10]
    lines = ["★ JOIN HUBS (pagerank top 10)"]
    for n, _ in hubs:
        succ = ", ".join(sorted(g.successors(n)))
        lines.append(f"{n} → {succ if succ else '(leaf)'}")
    return "\n".join(lines)

def schema_summary_string(df_cols: pd.DataFrame) -> str:
    lines = []
    for tbl, g in df_cols.groupby("table_name"):
        cols = ", ".join(f"{r.column_name} ({r.data_type})" for r in g.itertuples())
        lines.append(f"TABLE {tbl}: {cols}")
    return "\n".join(lines)

# ---------------- Helpers ----------------
def try_json_loads(txt: str):
    try:
        return json.loads(txt)
    except Exception:
        return None

def clean_sql(txt: str) -> str:
    if not txt:
        return ""
    s = txt.strip()
    # 코드펜스 제거
    s = re.sub(r"^```[a-zA-Z]*", "", s)
    s = re.sub(r"```$", "", s)
    s = s.strip()
    # 끝 세미콜론 보장
    if not s.endswith(";"):
        s += ";"
    return s

READ_ONLY_ERROR = "Only SELECT/CTE queries are allowed."

def validate_sql(sql: str, allowed_tables: list[str], schema_prefix="") -> tuple[bool, str]:
    """⭐ NEW: sqlglot AST 검증."""
    try:
        parsed = sqlglot.parse_one(sql)
    except sqlglot.errors.ParseError as e:
        return False, f"Parse error: {e}"
    if parsed.find(sqlglot.exp.Insert) or parsed.find(sqlglot.exp.Update):
        return False, READ_ONLY_ERROR
    tables = {t.name for t in parsed.find_all(sqlglot.exp.Table)}
    bare = {t.replace(schema_prefix, "") for t in tables}
    if not bare.issubset(set(allowed_tables)):
        return False, f"Unexpected tables: {bare - set(allowed_tables)}"
    return True, ""
# ---------------- State ----------------
@dataclass
class SQLState:
    question: str
    schema_text: str
    selected_schema_text: str = ""
    selected_tables: List[str] = field(default_factory=list)
    sub_questions: List[str] = field(default_factory=list)
    sub_sqls: List[str] = field(default_factory=list)
    sql: str = ""
    result_preview: str = ""
    error: str = ""
    try_count: int = 0
    critic: str = "" # ⭐ NEW: 실행 후 메트릭 저장
    schema_name: str = ""
    schema_prefix: str = ""
   # ⭐ NEW: result_json 필드 추가
    result_json: Optional[str] = None # JSON 문자열을 저장할 필드
    result_data_id: Optional[str] = None # 파일 ID를 저장할 필드 (Streamlit 연동용)

# ---------------- Prompts ----------------
SELECTOR_SYS = (
    "You are a database schema selector. Given a user question and the full schema summary, "
    "choose the MINIMAL set of tables strictly required to answer it. Prefer join keys. "
    'Return JSON: {"tables": ["..."], "rationale": "..."}'
)

SELECTOR_USER = (
    "Question:\n{q}\n\nFull Schema Summary:\n{schema}\n\n"
    "Return JSON with keys: tables, rationale."
)

DECOMP_SYS = (
    "You are a senior data analyst. Design a single, complete PostgreSQL query.\n"
    "Always schema-qualify tables with the prefix \"{prefix}\".\n"
    "Return STRICT JSON with EXACT keys: {{\"sub_questions\": [], \"sub_sqls\": [], \"final_sql\": \"\"}}.\n"
    "The final_sql MUST be a single executable query (no markdown, no explanation), and end with a semicolon.\n"
    "Do not omit GROUP BY when aggregates are used. Limit result rows if not an aggregate."
)


DECOMP_USER = (
    "Question: {q}\n\nRelevant Schema (summary):\n{schema}\n\nConstraints:\n"
    "- Use LEFT JOIN when rows might be missing.\n"
    "- Use explicit GROUP BY for aggregates.\n"
    "- Limit to 100 rows unless aggregation.\n"
    "- Timestamps are UTC; convert only if explicitly asked.\n"
    "Return JSON."
)

REFINER_SYS = (
    "You are a SQL mechanic. Given a user's question, the database schema summary, "
    "a PostgreSQL SQL query that failed, and the specific error message, your task is to fix the SQL.\n"
    "Carefully analyze the provided error message to understand the exact problem. "
    "Keep the original intent of the query. "
    "Return ONLY the complete, corrected PostgreSQL SQL query. Do not include any markdown formatting, explanations, or extra text. "
    "The corrected SQL query must end with a semicolon."
)

REFINER_USER_TMPL = (
    "Question: {q}\nSchema Summary:\n{schema}\n\nPrevious SQL:\n{sql}\n\nError:\n{err}\n\n"
    "Return corrected SQL only."
)

FORMATTER_SYS = (
    "You are a data reporter. Given the user question, final SQL and a small result sample, "
    "write a short 2-3 sentence summary in Korean, then output a compact markdown table header."
)

FORMATTER_USER = (
    "Question: {q}\nSQL:\n{sql}\n\nPreview (first rows):\n{preview}\n"
)

REGEN_SYS = (
    "You are a SQL expert. Generate ONE complete PostgreSQL query for the user's question.\n"
    "Use the provided schema summary. Always schema-qualify with prefix \"{prefix}\".\n"
    "Return ONLY SQL, no markdown, no explanation. End with a semicolon."
)
REGEN_USER = "Question: {q}\nSchema Summary:\n{schema}\n"
CRITIC_SYS = (
    "You are a SQL critic. If rows>1000 OR null_ratio>0.5, suggest how to LIMIT rows or handle NULLs. "
    "Return 'pass' if no change needed."
)
CRITIC_USER = (
    "Question: {q}\nSQL:\n{sql}\nMetrics: rows={rows}, null_ratio={null_ratio:.2%}\nSchema:\n{schema}"
)
# ---------------- Nodes ----------------
# ⭐ NEW node_validator

def node_validator(st: SQLState) -> SQLState:
    if st.sql:
        ok, msg = validate_sql(st.sql, st.selected_tables or [], st.schema_prefix)
        if not ok:
            st.error = "Validator: " + msg
    return st

def node_selector(st: SQLState, llm: GeminiClient) -> SQLState:
    raw = llm.generate(
        SELECTOR_USER.format(q=st.question, schema=st.schema_text),
        system_instruction=SELECTOR_SYS,
        json_mode=True
    )
    if DEBUG:
        print("🔎 [selector] raw:", raw[:600])
    data = try_json_loads(raw) or {"tables": [], "rationale": "fallback"}
    st.selected_tables = data.get("tables", []) or []
    if st.selected_tables:
        lines = []
        for line in st.schema_text.splitlines():
            if not line.startswith("TABLE "):
                continue
            tbl = line.split()[1].rstrip(":")
            if tbl in st.selected_tables:
                lines.append(line)
        st.selected_schema_text = "\n".join(lines) if lines else st.schema_text
    else:
        st.selected_schema_text = st.schema_text
    return st

def node_decomposer(st: SQLState, llm: GeminiClient) -> SQLState:
    sys = DECOMP_SYS.format(prefix=st.schema_prefix)
    raw = llm.generate(
        DECOMP_USER.format(q=st.question, schema=st.selected_schema_text or st.schema_text),
        system_instruction=sys,
        json_mode=True
    )
    if DEBUG:
        print("🔎 [decomposer] raw:", raw[:1200])
    data = try_json_loads(raw)
    if not data:
        st.error = "LLM JSON parsing failed (decomposer)."
        return st
    st.sub_questions = data.get("sub_questions", [])
    st.sub_sqls = data.get("sub_sqls", [])
    st.sql = clean_sql(data.get("final_sql", ""))
    if not st.sql:
        st.error = "Final SQL missing."
    return st

# executor ⭐ CHG: critic 메트릭 계산 추가

# executor ⭐ CHG: critic 메트릭 계산 추가, result_json 저장 추가

def execute_sql(engine: Engine, sql: str):
    if not sql.lower().lstrip().startswith(("select", "with")):
        raise ValueError(READ_ONLY_ERROR)
    with engine.connect() as conn:
        return pd.read_sql_query(text(sql), conn)

def node_executor(st: SQLState, engine: Engine) -> SQLState:
    if st.sql and not st.error:
        try:
            df = execute_sql(engine, st.sql)
            rows = len(df)
            null_ratio = df.isna().mean().mean() if rows else 0
            st.critic = f"rows={rows}, null_ratio={null_ratio:.3f}"
            st.result_preview = df.head(100).to_markdown(index=False) if not df.empty else "| (결과 없음) |"

            # 🚨 중요: 여기에 이 라인을 추가하거나 확인하세요!
            # DataFrame이 비어있지 않을 경우 JSON으로 변환하여 저장
            if not df.empty:
                st.result_json = df.to_json(orient='records', force_ascii=False)
            else:
                st.result_json = None # 데이터가 없으면 None으로 설정

            st.result_data_id = None # Streamlit 연동 시 사용되므로 여기서는 None으로 유지

        except Exception as e:
            st.error = str(e)
    return st

# ⭐ NEW node_critic

def node_critic(st: SQLState, llm: GeminiClient) -> SQLState:
    if st.error: return st
    m = re.search(r"rows=(\d+), null_ratio=([0-9.]+)", st.critic)
    rows, nr = (int(m.group(1)), float(m.group(2))) if m else (0, 0)
    if rows <= 1000 and nr <= 0.5:
        return st
    suggestion = llm.generate(CRITIC_USER.format(q=st.question, sql=st.sql, rows=rows, null_ratio=nr, schema=st.selected_schema_text),
                              system_instruction=CRITIC_SYS)
    if suggestion.strip().lower() != "pass":
        st.error = "Critic: " + suggestion.strip()
    return st
#---------------
def node_refiner(st: SQLState, llm: GeminiClient) -> SQLState:
    if not st.error:
        return st
    fixed = llm.generate(
        REFINER_USER_TMPL.format(
            q=st.question,
            schema=st.selected_schema_text or st.schema_text,
            sql=st.sql,
            err=st.error
        ),
        system_instruction=REFINER_SYS,
        json_mode=False
    )
    if DEBUG:
        print("🔧 [refiner] fixed SQL:", fixed[:1200])
    st.sql = clean_sql(fixed)
    st.try_count += 1
    st.error = ""          # ⭐ 반드시 초기화
    return st

def node_regenerate(st: SQLState, llm: GeminiClient) -> SQLState:
    sql = llm.generate(
        REGEN_USER.format(q=st.question, schema=st.selected_schema_text or st.schema_text),
        system_instruction=REGEN_SYS.format(prefix=st.schema_prefix),
        json_mode=False
    )
    if DEBUG:
        print("🧩 [regen] sql:", sql[:1200])
    st.sql = clean_sql(sql)
    return st

def node_formatter(st: SQLState, llm: GeminiClient) -> SQLState:
    summary = llm.generate(
        FORMATTER_USER.format(q=st.question, sql=st.sql, preview=st.result_preview),
        system_instruction=FORMATTER_SYS
    )
    st.result_preview = summary + "\n\n" + st.result_preview
    return st

# ---------------- Graph build ----------------
def build_graph(engine: Engine, model: str):
    llm = GeminiClient(model)
    g = StateGraph(SQLState)

    g.add_node("selector",   lambda s: node_selector(s, llm))
    g.add_node("decomposer", lambda s: node_decomposer(s, llm))
    g.add_node("validator",  node_validator)
    g.add_node("executor",   lambda s: node_executor(s, engine))
    g.add_node("critic",     lambda s: node_critic(s, llm))
    g.add_node("refiner",    lambda s: node_refiner(s, llm))
    g.add_node("regen",      lambda s: node_regenerate(s, llm))
    g.add_node("formatter",  lambda s: node_formatter(s, llm))

    g.set_entry_point("selector")
    g.add_edge("selector", "decomposer")
    g.add_conditional_edges("decomposer", lambda s: "validator" if s.sql else "regen", {"validator": "validator", "regen": "regen"})
    g.add_edge("regen", "validator")
    g.add_conditional_edges("validator", lambda s: "refiner" if s.error else "executor", {"refiner": "refiner", "executor": "executor"})
    g.add_conditional_edges(
        "executor",
        # 실행 오류가 있고, refiner 시도 횟수가 2 미만일 경우 refiner로 이동. 그렇지 않으면 critic으로 이동.
        lambda s: "refiner" if s.error and s.try_count < 2 else "critic",
        {"refiner": "refiner", "critic": "critic"}
    )
    g.add_conditional_edges(
        "critic",
        # critic이 오류를 제안했고 (예: 결과가 너무 많거나 NULL 비율이 높음),
        # refiner 총 시도 횟수가 3 미만일 경우 refiner로 이동.
        # 그렇지 않으면 (critic이 'pass'를 반환했거나, 수정 시도를 너무 많이 했거나) formatter로 이동.
        lambda s: "refiner" if s.error and s.try_count < 3 else "formatter",
        {"refiner": "refiner", "formatter": "formatter"}
    )
    # refiner 노드는 항상 validator로 돌아가서 수정된 SQL을 다시 검증.
    g.add_edge("refiner", "validator")
    g.add_edge("formatter", END)
    return g.compile()

# ---------------- Normalize result ----------------
def to_state(res, fallback: SQLState) -> SQLState:
    """invoke 반환값이 dict/객체 어느 쪽이든 SQLState로 정규화."""

    # LangGraph의 invoke 결과가 이미 SQLState 객체의 필드를 담은 딕셔너리 형태일 때
    if isinstance(res, dict):
        # SQLState의 필드 이름을 정의합니다.
        field_names = {f.name for f in dataclasses.fields(SQLState)}

        # res 딕셔너리에서 SQLState의 필드에 해당하는 값만 추출합니다.
        payload = {k: v for k, v in res.items() if k in field_names}

        # 만약 payload에 필수 필드(예: question, schema_text)가 없다면 fallback에서 채웁니다.
        # 이 시점에는 raw_result에 모든 필드가 있으므로, 이 부분은 거의 실행되지 않을 것입니다.
        # 하지만 방어적인 코드로서 유지합니다.
        for f in dataclasses.fields(SQLState):
            if f.name not in payload or payload[f.name] is None:
                if hasattr(fallback, f.name) and getattr(fallback, f.name) is not None:
                    payload[f.name] = getattr(fallback, f.name)
                elif f.default is not dataclasses.MISSING:
                    payload[f.name] = f.default
                elif getattr(f, "default_factory", dataclasses.MISSING) is not dataclasses.MISSING: # type: ignore
                    payload[f.name] = f.default_factory() # type: ignore

        return SQLState(**payload)

    # 만약 res가 이미 SQLState 객체라면 그대로 반환합니다.
    elif isinstance(res, SQLState):
        return res

    # 그 외의 경우 (예상치 못한 타입), fallback을 반환하고 경고 메시지를 출력합니다.
    print("경고: to_state 함수가 예상치 못한 타입의 결과를 받았습니다. Fallback을 사용합니다.")
    return fallback

# ---------------- Run ----------------
engine = make_engine(CONN_STR)

# 스키마 자동 탐지 (ecommerce 우선, 없으면 public)
schema_name = "ecommerce" if schema_exists(engine, "ecommerce") else "public"
tbls = list_tables(engine, schema_name)
cols = fetch_columns(engine, schema_name)

schema_text = schema_info = '''
# Schema: public
(이 스키마에는 테이블이 없습니다.)

# Schema: ecommerce
Table: coupon
    coupon_id (text) - 쿠폰의 고유 식별자입니다.
    product_category (text) (Nullable) - 쿠폰을 적용할 수 있는 상품 카테고리입니다.
    discount_rate (integer) (Nullable) - 쿠폰의 할인율(%)입니다.
    PK: coupon_id
    FK: (product_category) -> product_categories(product_category)

Table: customer_info
    customer_unique_id (text) - 고객의 고유 식별자입니다. (시스템 전체에서 유일)
    customer_gender (text) - 고객의 성별입니다. (남: male, 여: female)
    customer_age_group (text) - 고객의 연령대입니다. (형식: 20s, 30s, 40s)
    PK: customer_unique_id

Table: customers
    customer_id (text) - 주문과 같은 트랜잭션에서 사용되는 고객 식별자입니다.
    customer_unique_id (text) - customer_info와 연결되는 고객의 고유 식별자입니다.
    customer_zip_code_prefix (text) (Nullable) - 고객 주소의 우편번호 앞자리입니다.
    customer_city (text) (Nullable) - 고객이 거주하는 도시입니다.
    customer_state (text) (Nullable) - 고객이 거주하는 주(State)입니다.
    PK: customer_id
    FK: (customer_unique_id) -> customer_info(customer_unique_id)

Table: events
    event_id (text) - 이벤트의 고유 식별자입니다.
    product_id (text) - 이벤트와 관련된 상품의 식별자입니다.
    customer_unique_id (text) - 이벤트를 발생시킨 고객의 고유 식별자입니다.
    event_type (text) - 이벤트 유형입니다. (유형: 'view', 'cart', 'purchase')
    event_time (timestamp without time zone) - 이벤트가 발생한 시각입니다.
    customer_id (character varying) (Nullable) - 이벤트 발생 시점의 고객 임시 식별자입니다.
    category (character varying) (Nullable) - 이벤트와 관련된 카테고리입니다.
    PK: event_id
    FK: (product_id) -> products(product_id)
    FK: (customer_unique_id) -> customer_info(customer_unique_id)

Table: marketing_campaign
    campaign_id (text) - 마케팅 캠페인의 고유 식별자입니다.
    send_timestamp (timestamp without time zone) (Nullable) - 캠페인 메시지를 발송한 시각입니다.
    day_of_week (text) (Nullable) - 메시지를 발송한 요일입니다.
    campaign_type (text) (Nullable) - 캠페인 유형입니다. (category: 'general_product', 'promotion')
    message_format (text) (Nullable) - 카카오톡 플러스 메시지를 보낼 때 페이지를 어떤 형식으로 표시했는지 나타냅니다. (category: 'carousel', 'wide_image')
    visits (integer) (Nullable) - 전체 방문 수입니다.
    ad_conversions (integer) (Nullable) - 캠페인을 통한 구매 수입니다.
    total_conversions (integer) (Nullable) - 총 구매 수입니다.
    conversion_rate (real) (Nullable) - 구매전환율 (%) 입니다.
    PK: campaign_id

Table: order_items
    order_id (text) - 상품이 포함된 주문의 식별자입니다.
    product_id (text) - 주문된 상품의 식별자입니다.
    quantity (integer) - 주문된 상품의 수량입니다.
    seller_id (text) - 해당 상품을 판매한 판매자의 식별자입니다.
    shipping_limit_date (timestamp without time zone) - 판매자가 상품을 발송해야 하는 마감 시한입니다.
    price (double precision) - 상품 한 개의 가격입니다.
    freight_value (double precision) - 배송비입니다.
    PK: order_id, product_id
    FK: (order_id) -> orders(order_id)
    FK: (product_id) -> products(product_id)

Table: order_payments
    order_id (text) - 결제가 이루어진 주문의 식별자입니다.
    payment_sequential (integer) - 한 주문에 여러 결제 수단이 사용될 경우의 순서입니다.
    payment_type (text) (Nullable) - 결제 수단 유형입니다. (예: 'credit_card', 'boleto')
    payment_installments (integer) (Nullable) - 할부 개월 수입니다.
    payment_value (double precision) - 결제 금액입니다.
    PK: order_id, payment_sequential
    FK: (order_id) -> orders(order_id)

Table: order_reviews
    review_id (text) - 리뷰의 고유 식별자입니다.
    order_id (text) - 해당 리뷰가 작성된 주문의 식별자입니다.
    review_score (integer) (Nullable) - 고객이 부여한 리뷰 점수입니다. (1~5)
    review_comment_title (text) (Nullable) - 리뷰 제목입니다.
    review_comment_message (text) (Nullable) - 리뷰 내용입니다.
    review_creation_date (timestamp without time zone) (Nullable) - 리뷰가 작성된 시각입니다.
    review_answer_timestamp (timestamp without time zone) (Nullable) - 판매자가 리뷰에 답변한 시각입니다.
    PK: review_id, order_id
    FK: (order_id) -> orders(order_id)

Table: orders
    order_id (text) - 주문의 고유 식별자입니다.
    customer_id (text) - 주문을 한 고객의 식별자입니다.
    order_status (text) - 주문의 현재 상태입니다. (예: 'delivered', 'shipped')
    order_purchase_timestamp (timestamp without time zone) (Nullable) - 고객이 주문을 완료한 시각입니다.
    order_approved_at (timestamp without time zone) (Nullable) - 주문이 승인된 시각입니다.
    order_delivered_carrier_date (timestamp without time zone) (Nullable) - 상품이 배송업체에 전달된 시각입니다.
    order_delivered_customer_date (timestamp without time zone) (Nullable) - 상품이 고객에게 배송 완료된 시각입니다.
    order_estimated_delivery_date (timestamp without time zone) (Nullable) - 예상 배송 완료일입니다.
    PK: order_id
    FK: (customer_id) -> customers(customer_id)

Table: product_categories
    product_category (text) - 상품 카테고리의 영문 이름입니다.
    product_category_portugese (text) (Nullable) - 상품 카테고리의 포르투갈어 이름입니다.
    PK: product_category

Table: products
    product_id (text) - 상품의 고유 식별자입니다.
    product_category (text) (Nullable) - 상품이 속한 카테고리입니다.
    product_name_lenght (integer) (Nullable) - 상품 이름의 글자 수입니다.
    product_description_lenght (integer) (Nullable) - 상품 설명의 글자 수입니다.
    product_photos_qty (integer) (Nullable) - 상품 사진의 개수입니다.
    product_weight_g (integer) (Nullable) - 상품의 무게(g)입니다.
    product_length_cm (integer) (Nullable) - 상품의 길이(cm)입니다.
    product_height_cm (integer) (Nullable) - 상품의 높이(cm)입니다.
    product_width_cm (integer) (Nullable) - 상품의 너비(cm)입니다.
    PK: product_id
    FK: (product_category) -> product_categories(product_category)

Table: promotion
    promotion_id (text) - 프로모션의 고유 식별자입니다.
    start_date (timestamp without time zone) (Nullable) - 프로모션 시작일입니다.
    end_date (timestamp without time zone) (Nullable) - 프로모션 종료일입니다.
    PK: promotion_id

Table: promotion_coupon_mapping
    promotion_id (text) - 프로모션 식별자입니다. (M:N 관계 연결 테이블)
    coupon_id (text) - 쿠폰 식별자입니다. (M:N 관계 연결 테이블)
    PK: promotion_id, coupon_id
    FK: (promotion_id) -> promotion(promotion_id)
    FK: (coupon_id) -> coupon(coupon_id)
'''
schema_prefix = f"{schema_name}."

if DEBUG:
    print(f"✅ Using schema: {schema_name}, tables={len(tbls)}")

app = build_graph(engine, GEMINI_MODEL)



✅ Using schema: ecommerce, tables=13


In [ ]:
import json
import pandas as pd
from IPython.display import Markdown, display, HTML
import os
import io

# --- 기존 LangGraph 에이전트 빌드 및 실행 부분 (생략 - 위에 있는 코드 그대로) ---
# ... (GEMINI_MODEL, engine, schema_text, app 등 정의 및 초기화) ...

question = "리뷰 평점이 4점 이상이고 배송이 5일 이내에 완료된 주문의 평균 결제 금액을 카테고리별로 알려주세요."
init = SQLState(
    question=question,
    schema_text=schema_text,
    schema_name=schema_name,
    schema_prefix=schema_prefix,
)

raw_result = app.invoke(init)
state = to_state(raw_result, init)

print("\n🪵 선택된 테이블:", state.selected_tables)
print("🪵 생성된 sub-questions:", state.sub_questions)
print("🪵 생성된 sub-sqls:", state.sub_sqls)
print("🪵 생성된 SQL:", state.sql)
print("🪵 에러 메시지:", state.error)

from IPython.display import Markdown, display
display(Markdown("### ✅ 결과 미리보기\n" + (state.result_preview or "_(출력 없음)_")))

result_data_json = None
if state.result_json:
    try:
        result_data_json = json.loads(state.result_json)
        print("\n--- 시각화를 위한 추출된 JSON 데이터 ---")
        print(json.dumps(result_data_json, indent=2, ensure_ascii=False))
    except json.JSONDecodeError as e:
        print(f"JSON 데이터 디코딩 오류: {e}")
        result_data_json = None
else:
    print("\n--- 시각화할 JSON 데이터 없음 ---")
    if state.error:
        print(f"에이전트 오류: {state.error}")
    elif state.result_preview == "| (결과 없음) |":
        print("SQL 쿼리 결과가 비어 있습니다.")

# --- Gemini 클라이언트 초기화 ---
llm_code_gen = GeminiClient(model=GEMINI_MODEL)



# --- 시각화 코드 생성을 위한 프롬프트 구성 함수 ---
def get_visualization_prompt(original_question: str, sql_query: str, data: list[dict]) -> str:
    if not data:
        return "데이터가 없어 시각화 코드를 생성할 수 없습니다."

    df = pd.DataFrame(data)
    chart_type_hint = suggest_chart_type(df, original_question)
    # 데이터 요약 정보 생성
    summary_parts = []
    summary_parts.append(f"총 {len(df)}개의 행이 있습니다.")

    for col in df.columns:
        col_type = pd.api.types.infer_dtype(df[col])
        num_unique = df[col].nunique()
        summary_parts.append(f"- '{col}' 컬럼: 타입='{col_type}', 고유값={num_unique}개.")
        if pd.api.types.is_numeric_dtype(df[col]):
            summary_parts.append(f"  (최소={df[col].min():.2f}, 최대={df[col].max():.2f}, 평균={df[col].mean():.2f})")

    data_summary = "\n".join(summary_parts)

    # 시각화 라이브러리 및 요구사항 명시
    prompt = f"""
    📌 반드시 지켜야 할 규칙
    1. 숫자·문자 데이터를 새로 작성하지 않는다(하드코딩 금지).
    2. 이미 정의된 `data` 변수를 그대로 사용 → `pd.DataFrame(data)`로만 변환.
    3. 필요한 경우 컬럼 요약(**값 제외**)만 참고한다.

    **추천 차트 유형 힌트 (자동 분석 기반):**
    "{chart_type_hint}" 차트가 적절하다고 판단됩니다.
    이 힌트를 참고하여 차트 유형을 선택하되, 데이터 분석 결과와 질문의 의도에 따라 필요 시 다른 유형을 선택해도 됩니다.

    **사용자의 원래 질문 (비즈니스 맥락):**
    "{original_question}"

    **SQL 쿼리:**
    ```sql
    {sql_query}
    ```


    **쿼리 결과 컬럼 정보(값 제외)**
  {', '.join(df.columns)}  |  Row count = {len(df)}

    **데이터 요약 통계:**
    {data_summary}

    **시각화 코드 생성 요구사항:**
    1.  데이터프레임을 로드하는 코드 (`df = pd.DataFrame(data)`)부터 시작해주세요. `data` 변수에는 전체 JSON 데이터가 리스트 형태로 들어있다고 가정합니다.
    2.  데이터의 컬럼 이름, 데이터 타입, 그리고 사용자의 원래 질문을 고려하여 **가장 적절하다고 판단되는 차트 유형** (예: 막대 그래프, 선 그래프, 파이 차트, 산점도 등)을 선택해주세요.
    3.  차트 유형에 맞게 **X축/Y축 또는 라벨/값에 가장 적절한 컬럼을 자동으로 선택**해주세요.
    4.  차트 제목은 사용자의 질문과 데이터 내용을 반영하여 적절하게 생성해주세요.
    5.  축 라벨도 컬럼의 의미에 맞게 적절하게 지정해주세요.
    6.  생성된 차트(`fig`)를 `fig.show()`로 **화면에 표시하는 코드와 함께**, `fig.to_json()`으로 JSON 문자열로 변환한 후, `json.loads` 및 `json.dumps(indent=2)`를 사용하여 예쁘게 만든 JSON을 '{original_question[:20].replace(' ', '_')}_chart.json' 파일명으로 저장하는 코드까지 포함해주세요.
    7.  파일 저장 후에는 "차트 JSON이 '[파일명]' 파일로 저장되었습니다." 메시지를 출력해주세요.
    8.  모든 코드를 하나의 파이썬 코드 블록(` ```python `)으로 감싸서 반환해주세요.
    9.  어떤 SQL 쿼리가 들어오든 **항상** 시각화 코드를 생성하려고 시도해주세요. 만약 시각화에 적합하지 않은 데이터라고 판단되면, 왜 그런지 간략히 주석으로 설명하고 간단한 테이블 출력 코드라도 생성해주세요.
    """
    return prompt


def suggest_chart_type(df: pd.DataFrame, question: str) -> str:
    scores = {
        "bar": 0,
        "line": 0,
        "pie": 0,
        "sankey": 0,
        "scatter": 0,
    }

    q = question.lower()
    if any(x in q for x in ["추이", "시간", "월별", "요일", "변화", "시계열", "타임라인"]):
        scores["line"] += 3
    if any(x in q for x in ["비교", "가장 많은", "상위", "top", "개수"]):
        scores["bar"] += 3
    if any(x in q for x in ["비율", "퍼센트", "구성", "분포", "점유율"]):
        scores["pie"] += 3
    if any(x in q for x in ["흐름", "전환", "이동", "경로"]):
        scores["sankey"] += 3
    if any(x in q for x in ["상관", "관계", "산점도"]):
        scores["scatter"] += 3

    num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
    cat_cols = [c for c in df.columns if pd.api.types.is_object_dtype(df[c]) or df[c].dtype.name == "category"]

    if len(cat_cols) == 1 and len(num_cols) == 1:
        scores["bar"] += 2
        scores["pie"] += 1
    if len(cat_cols) == 2 and len(num_cols) == 1:
        scores["sankey"] += 2
    if len(num_cols) >= 2:
        scores["scatter"] += 2
    if "date" in " ".join(df.columns).lower() or df.index.dtype.name.startswith("datetime"):
        scores["line"] += 2

    return sorted(scores.items(), key=lambda x: -x[1])[0][0]

# --- 시각화 코드 생성 및 실행 ---
if result_data_json:
    prompt_for_code_gen = get_visualization_prompt(state.question, state.sql, result_data_json)

    print("\n--- Gemini에게 동적 시각화 코드 생성 요청 중 ---")
    generated_code = llm_code_gen.generate(prompt_for_code_gen)

    print("\n--- Gemini가 생성한 시각화 코드 ---")
    print(generated_code)

    try:
        # 기존: Gemini가 생성한 시각화 코드 실행
        cleaned_code = generated_code.replace("```python", "").replace("```", "").strip()
        data = result_data_json
        import plotly.express as px
        import plotly.io as pio
        pio.templates.default = "plotly_white"
        exec(cleaned_code, globals(), locals())
        print("\n--- 생성된 시각화 코드 실행 완료 ---")

    except Exception as e:
        print(f"\n⚠️ 생성된 코드 실행 중 오류 발생: {e}")
        print("🔁 fallback 차트 자동 추천 및 실행")

        # ✅ fallback 시각화
        try:
            df = pd.DataFrame(result_data_json)

            if df.shape[1] < 2:
                print("⚠️ fallback 시각화를 위한 컬럼이 부족합니다.")
                display(df.head())
            else:
                chart_type = suggest_chart_type(df, state.question)
                print("🎯 추천된 fallback 차트 유형:", chart_type)

                import plotly.express as px
                import plotly.graph_objects as go

                fig = None
                if chart_type == "bar":
                    fig = px.bar(df, x=df.columns[0], y=df.columns[1])
                elif chart_type == "line":
                    fig = px.line(df, x=df.columns[0], y=df.columns[1])
                elif chart_type == "pie":
                    fig = px.pie(df, names=df.columns[0], values=df.columns[1])
                elif chart_type == "scatter":
                    fig = px.scatter(df, x=df.columns[0], y=df.columns[1])
                elif chart_type == "sankey":
                    print("⚠️ Sankey 차트는 자동 구성 미지원입니다. (고도화 필요)")

                if fig:
                    fig.show()
                    print("✅ fallback 차트 실행 완료")

        except Exception as ee:
            print(f"❌ fallback 시각화도 실패: {ee}")

else:
    print("\nJSON 데이터가 없으므로 시각화 코드 생성을 건너뛴니다.")

🔎 [selector] raw: {
  "tables": [
    "order_reviews",
    "orders",
    "order_payments",
    "order_items",
    "products"
  ],
  "rationale": "To filter by '리뷰 평점이 4점 이상' (review score 4 or higher), the 'order_reviews' table is needed. To filter by '배송이 5일 이내에 완료된 주문' (delivery within 5 days), the 'orders' table is required to calculate the difference between 'order_purchase_timestamp' and 'order_delivered_customer_date'. To find the '평균 결제 금액' (average payment amount), the 'order_payments' table is necessary for 'payment_value'. To group by '카테고리별로' (by category), 'order_items' is needed to link orders to p
🔎 [decomposer] raw: {
  "sub_questions": [
    "리뷰 평점이 4점 이상인 주문을 식별합니다.",
    "배송이 5일 이내에 완료된 주문을 식별합니다.",
    "각 주문의 결제 금액과 해당 주문에 포함된 상품의 카테고리를 연결합니다.",
    "위 조건을 모두 만족하는 주문들의 결제 금액을 카테고리별로 그룹화하여 평균을 계산합니다."
  ],
  "sub_sqls": [
    "SELECT order_id FROM ecommerce.order_reviews WHERE review_score >= 4;",
    "SELECT order_id FROM ecommerce.orders WHERE order_delivered_custom

### ✅ 결과 미리보기
리뷰 평점이 4점 이상이고 배송이 5일 이내에 완료된 주문들의 카테고리별 평균 결제 금액을 분석했습니다. 이 결과는 고객 만족도가 높고 배송이 빠른 주문의 카테고리별 평균 결제액을 보여줍니다.

| 카테고리 | 평균 결제 금액 |
|:---|---:|

| product_category                        |   average_payment_value |
|:----------------------------------------|------------------------:|
| agro_industry_and_commerce              |                231.435  |
| air_conditioning                        |                420.257  |
| art                                     |                101.592  |
| arts_and_craftmanship                   |                 62.4167 |
| audio                                   |                 65.1435 |
| auto                                    |                105.263  |
| baby                                    |                129.011  |
| bed_bath_table                          |                127.432  |
| books_general_interest                  |                113.316  |
| books_imported                          |                 92.7123 |
| books_technical                         |                 69.5763 |
| christmas_supplies                      |                 53.93   |
| cine_photo                              |                 62.1585 |
| computers                               |               1473.04   |
| computers_accessories                   |                 93.6932 |
| consoles_games                          |                124.897  |
| construction_tools_construction         |                205.885  |
| construction_tools_lights               |                189.311  |
| construction_tools_safety               |                285.333  |
| cool_stuff                              |                150.774  |
| costruction_tools_garden                |                118.639  |
| costruction_tools_tools                 |                363.283  |
| diapers_and_hygiene                     |                119.29   |
| drinks                                  |                 64.8586 |
| dvds_blu_ray                            |                 33.3567 |
| electronics                             |                 62.233  |
| fashio_female_clothing                  |                 50.88   |
| fashion_bags_accessories                |                 80.4521 |
| fashion_childrens_clothes               |                113.653  |
| fashion_male_clothing                   |                 58.6025 |
| fashion_shoes                           |                153.539  |
| fashion_sport                           |                 86.7    |
| fashion_underwear_beach                 |                 73.74   |
| fixed_telephony                         |                123.52   |
| flowers                                 |                 53.96   |
| food                                    |                 77.3878 |
| food_drink                              |                 80.9181 |
| furniture_bedroom                       |                512.132  |
| furniture_decor                         |                118.282  |
| furniture_living_room                   |                125.497  |
| furniture_mattress_and_upholstery       |                123.118  |
| garden_tools                            |                144.641  |
| health_beauty                           |                126.469  |
| home_appliances                         |                 96.0151 |
| home_appliances_2                       |                552.792  |
| home_comfort_2                          |                 86.235  |
| home_confort                            |                179.04   |
| home_construction                       |                152.789  |
| housewares                              |                101.408  |
| industry_commerce_and_business          |                175.59   |
| kitchen_dining_laundry_garden_furniture |                150.42   |
| la_cuisine                              |                186.945  |
| luggage_accessories                     |                173.406  |
| market_place                            |                 76.5378 |
| music                                   |                179.542  |
| musical_instruments                     |                216.447  |
| office_furniture                        |                277.717  |
| party_supplies                          |                 35.688  |
| perfumery                               |                128.81   |
| pet_shop                                |                146.779  |
| signaling_and_security                  |                 99.3047 |
| small_appliances                        |                284.226  |
| small_appliances_home_oven_and_coffee   |                751.208  |
| sports_leisure                          |                114.819  |
| stationery                              |                 99.6068 |
| tablets_printing_image                  |                119.234  |
| telephony                               |                 53.4167 |
| toys                                    |                133.729  |
| watches_gifts                           |                187.997  |
|                                         |                133.838  |


--- 시각화를 위한 추출된 JSON 데이터 ---
[
  {
    "product_category": "agro_industry_and_commerce",
    "average_payment_value": 231.4347058824
  },
  {
    "product_category": "air_conditioning",
    "average_payment_value": 420.2568421053
  },
  {
    "product_category": "art",
    "average_payment_value": 101.5918181818
  },
  {
    "product_category": "arts_and_craftmanship",
    "average_payment_value": 62.4166666667
  },
  {
    "product_category": "audio",
    "average_payment_value": 65.1434615385
  },
  {
    "product_category": "auto",
    "average_payment_value": 105.2629081633
  },
  {
    "product_category": "baby",
    "average_payment_value": 129.0106122449
  },
  {
    "product_category": "bed_bath_table",
    "average_payment_value": 127.4321003135
  },
  {
    "product_category": "books_general_interest",
    "average_payment_value": 113.3160784314
  },
  {
    "product_category": "books_imported",
    "average_payment_value": 92.7123076923
  },
  {
    "product_category": "boo

차트 JSON이 '리뷰_평점이_4점_이상이고_배송이_5_chart.json' 파일로 저장되었습니다.

--- 생성된 시각화 코드 실행 완료 ---


In [ ]:
import plotly.io as pio
import json

# 저장된 JSON 파일 경로
file_path = "/content/리뷰_평점이_4점_이상이고_배송이_5_chart.json"

# JSON 파일 읽기
with open(file_path, "r", encoding="utf-8") as f:
    chart_json = json.load(f)

# Plotly Figure 객체로 복원
fig = pio.from_json(json.dumps(chart_json))

# 시각화 표시
fig.show()


In [ ]:
import json
import pandas as pd
from IPython.display import Markdown, display, HTML
import os
import io

# --- 기존 LangGraph 에이전트 빌드 및 실행 부분 (생략 - 위에 있는 코드 그대로) ---
# ... (GEMINI_MODEL, engine, schema_text, app 등 정의 및 초기화) ...

question = "20대와 30대 여성 고객이 구매한 제품 중 평균 구매 금액이 가장 높은 5개 영어 카테고리를 알려주세요."
init = SQLState(
    question=question,
    schema_text=schema_text,
    schema_name=schema_name,
    schema_prefix=schema_prefix,
)

raw_result = app.invoke(init)
state = to_state(raw_result, init)

print("\n🪵 선택된 테이블:", state.selected_tables)
print("🪵 생성된 sub-questions:", state.sub_questions)
print("🪵 생성된 sub-sqls:", state.sub_sqls)
print("🪵 생성된 SQL:", state.sql)
print("🪵 에러 메시지:", state.error)

from IPython.display import Markdown, display
display(Markdown("### ✅ 결과 미리보기\n" + (state.result_preview or "_(출력 없음)_")))

result_data_json = None
if state.result_json:
    try:
        result_data_json = json.loads(state.result_json)
        print("\n--- 시각화를 위한 추출된 JSON 데이터 ---")
        print(json.dumps(result_data_json, indent=2, ensure_ascii=False))
    except json.JSONDecodeError as e:
        print(f"JSON 데이터 디코딩 오류: {e}")
        result_data_json = None
else:
    print("\n--- 시각화할 JSON 데이터 없음 ---")
    if state.error:
        print(f"에이전트 오류: {state.error}")
    elif state.result_preview == "| (결과 없음) |":
        print("SQL 쿼리 결과가 비어 있습니다.")

# --- Gemini 클라이언트 초기화 ---
llm_code_gen = GeminiClient(model=GEMINI_MODEL)



# --- 시각화 코드 생성을 위한 프롬프트 구성 함수 ---
def get_visualization_prompt(original_question: str, sql_query: str, data: list[dict]) -> str:
    if not data:
        return "데이터가 없어 시각화 코드를 생성할 수 없습니다."

    df = pd.DataFrame(data)

    # 데이터 요약 정보 생성
    summary_parts = []
    summary_parts.append(f"총 {len(df)}개의 행이 있습니다.")

    for col in df.columns:
        col_type = pd.api.types.infer_dtype(df[col])
        num_unique = df[col].nunique()
        summary_parts.append(f"- '{col}' 컬럼: 타입='{col_type}', 고유값={num_unique}개.")
        if pd.api.types.is_numeric_dtype(df[col]):
            summary_parts.append(f"  (최소={df[col].min():.2f}, 최대={df[col].max():.2f}, 평균={df[col].mean():.2f})")

    data_summary = "\n".join(summary_parts)

    # 시각화 라이브러리 및 요구사항 명시
    prompt = f"""
    아래는 사용자의 원래 질문, SQL 쿼리, 그리고 해당 쿼리 실행 결과의 JSON 데이터입니다.
    이 정보를 바탕으로 파이썬의 **Plotly Express** 라이브러리를 사용하여 가장 적합한 시각화 코드를 생성해주세요.
    ⚠️ 중요한 규칙: 아래 JSON 데이터의 수치를 절대 변경하지 마세요. 있는 그대로 사용하세요.
    **사용자의 원래 질문 (비즈니스 맥락):**
    "{original_question}"

    **SQL 쿼리:**
    ```sql
    {sql_query}
    ```

    **쿼리 결과 (JSON 데이터 예시 - 상위 5개 행만):**
    ```json
    {json.dumps(data[:5], indent=2, ensure_ascii=False)}
    ```

    **데이터 요약 통계:**
    {data_summary}

    **시각화 코드 생성 요구사항:**
    1.  데이터프레임을 로드하는 코드 (`df = pd.DataFrame(data)`)부터 시작해주세요. `data` 변수에는 전체 JSON 데이터가 리스트 형태로 들어있다고 가정합니다.
    2.  데이터의 컬럼 이름, 데이터 타입, 그리고 사용자의 원래 질문을 고려하여 **가장 적절하다고 판단되는 차트 유형** (예: 막대 그래프, 선 그래프, 파이 차트, 산점도 등)을 선택해주세요.
    3.  차트 유형에 맞게 **X축/Y축 또는 라벨/값에 가장 적절한 컬럼을 자동으로 선택**해주세요.
    4.  차트 제목은 사용자의 질문과 데이터 내용을 반영하여 적절하게 생성해주세요.
    5.  축 라벨도 컬럼의 의미에 맞게 적절하게 지정해주세요.
    6.  생성된 차트(`fig`)를 `fig.show()`로 **화면에 표시하는 코드와 함께**, `fig.to_json()`으로 JSON 문자열로 변환한 후, `json.loads` 및 `json.dumps(indent=2)`를 사용하여 예쁘게 만든 JSON을 '{original_question[:20].replace(' ', '_')}_chart.json' 파일명으로 저장하는 코드까지 포함해주세요.
    7.  파일 저장 후에는 "차트 JSON이 '[파일명]' 파일로 저장되었습니다." 메시지를 출력해주세요.
    8.  모든 코드를 하나의 파이썬 코드 블록(` ```python `)으로 감싸서 반환해주세요.
    9.  어떤 SQL 쿼리가 들어오든 **항상** 시각화 코드를 생성하려고 시도해주세요. 만약 시각화에 적합하지 않은 데이터라고 판단되면, 왜 그런지 간략히 주석으로 설명하고 간단한 테이블 출력 코드라도 생성해주세요.
    """
    return prompt


def suggest_chart_type(df: pd.DataFrame, question: str) -> str:
    scores = {
        "bar": 0,
        "line": 0,
        "pie": 0,
        "sankey": 0,
        "scatter": 0,
    }

    q = question.lower()
    if any(x in q for x in ["추이", "시간", "월별", "요일", "변화", "시계열", "타임라인"]):
        scores["line"] += 3
    if any(x in q for x in ["비교", "가장 많은", "상위", "top", "개수"]):
        scores["bar"] += 3
    if any(x in q for x in ["비율", "퍼센트", "구성", "분포", "점유율"]):
        scores["pie"] += 3
    if any(x in q for x in ["흐름", "전환", "이동", "경로"]):
        scores["sankey"] += 3
    if any(x in q for x in ["상관", "관계", "산점도"]):
        scores["scatter"] += 3

    num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
    cat_cols = [c for c in df.columns if pd.api.types.is_object_dtype(df[c]) or df[c].dtype.name == "category"]

    if len(cat_cols) == 1 and len(num_cols) == 1:
        scores["bar"] += 2
        scores["pie"] += 1
    if len(cat_cols) == 2 and len(num_cols) == 1:
        scores["sankey"] += 2
    if len(num_cols) >= 2:
        scores["scatter"] += 2
    if "date" in " ".join(df.columns).lower() or df.index.dtype.name.startswith("datetime"):
        scores["line"] += 2

    return sorted(scores.items(), key=lambda x: -x[1])[0][0]

# --- 시각화 코드 생성 및 실행 ---
if result_data_json:
    prompt_for_code_gen = get_visualization_prompt(state.question, state.sql, result_data_json)

    print("\n--- Gemini에게 동적 시각화 코드 생성 요청 중 ---")
    generated_code = llm_code_gen.generate(prompt_for_code_gen)

    print("\n--- Gemini가 생성한 시각화 코드 ---")
    print(generated_code)

    try:
        # 기존: Gemini가 생성한 시각화 코드 실행
        cleaned_code = generated_code.replace("```python", "").replace("```", "").strip()
        data = result_data_json
        import plotly.express as px
        import plotly.io as pio
        pio.templates.default = "plotly_white"
        exec(cleaned_code, globals(), locals())
        print("\n--- 생성된 시각화 코드 실행 완료 ---")

    except Exception as e:
        print(f"\n⚠️ 생성된 코드 실행 중 오류 발생: {e}")
        print("🔁 fallback 차트 자동 추천 및 실행")

        # ✅ fallback 시각화
        try:
            df = pd.DataFrame(result_data_json)

            if df.shape[1] < 2:
                print("⚠️ fallback 시각화를 위한 컬럼이 부족합니다.")
                display(df.head())
            else:
                chart_type = suggest_chart_type(df, state.question)
                print("🎯 추천된 fallback 차트 유형:", chart_type)

                import plotly.express as px
                import plotly.graph_objects as go

                fig = None
                if chart_type == "bar":
                    fig = px.bar(df, x=df.columns[0], y=df.columns[1])
                elif chart_type == "line":
                    fig = px.line(df, x=df.columns[0], y=df.columns[1])
                elif chart_type == "pie":
                    fig = px.pie(df, names=df.columns[0], values=df.columns[1])
                elif chart_type == "scatter":
                    fig = px.scatter(df, x=df.columns[0], y=df.columns[1])
                elif chart_type == "sankey":
                    print("⚠️ Sankey 차트는 자동 구성 미지원입니다. (고도화 필요)")

                if fig:
                    fig.show()
                    print("✅ fallback 차트 실행 완료")

        except Exception as ee:
            print(f"❌ fallback 시각화도 실패: {ee}")

else:
    print("\nJSON 데이터가 없으므로 시각화 코드 생성을 건너뛴니다.")

KeyboardInterrupt: 

In [ ]:
import json
import pandas as pd
from IPython.display import Markdown, display, HTML
import os
import io

# --- 기존 LangGraph 에이전트 빌드 및 실행 부분 (생략 - 위에 있는 코드 그대로) ---
# ... (GEMINI_MODEL, engine, schema_text, app 등 정의 및 초기화) ...

question = "월별(order_purchase_timestamp 기준)로 가장 많은 리뷰 점수(review_score) 평균을 기록한 상품 카테고리는 무엇이며, 해당 카테고리의 월별 평균 리뷰 점수와 해당 월의 전체 평균 리뷰 점수를 함께 보여주세요."
init = SQLState(
    question=question,
    schema_text=schema_text,
    schema_name=schema_name,
    schema_prefix=schema_prefix,
)

raw_result = app.invoke(init)
state = to_state(raw_result, init)

print("\n🪵 선택된 테이블:", state.selected_tables)
print("🪵 생성된 sub-questions:", state.sub_questions)
print("🪵 생성된 sub-sqls:", state.sub_sqls)
print("🪵 생성된 SQL:", state.sql)
print("🪵 에러 메시지:", state.error)

from IPython.display import Markdown, display
display(Markdown("### ✅ 결과 미리보기\n" + (state.result_preview or "_(출력 없음)_")))

result_data_json = None
if state.result_json:
    try:
        result_data_json = json.loads(state.result_json)
        print("\n--- 시각화를 위한 추출된 JSON 데이터 ---")
        print(json.dumps(result_data_json, indent=2, ensure_ascii=False))
    except json.JSONDecodeError as e:
        print(f"JSON 데이터 디코딩 오류: {e}")
        result_data_json = None
else:
    print("\n--- 시각화할 JSON 데이터 없음 ---")
    if state.error:
        print(f"에이전트 오류: {state.error}")
    elif state.result_preview == "| (결과 없음) |":
        print("SQL 쿼리 결과가 비어 있습니다.")

# --- Gemini 클라이언트 초기화 ---
llm_code_gen = GeminiClient(model=GEMINI_MODEL)



# --- 시각화 코드 생성을 위한 프롬프트 구성 함수 ---
def get_visualization_prompt(original_question: str, sql_query: str, data: list[dict]) -> str:
    if not data:
        return "데이터가 없어 시각화 코드를 생성할 수 없습니다."

    df = pd.DataFrame(data)

    # 데이터 요약 정보 생성
    summary_parts = []
    summary_parts.append(f"총 {len(df)}개의 행이 있습니다.")

    for col in df.columns:
        col_type = pd.api.types.infer_dtype(df[col])
        num_unique = df[col].nunique()
        summary_parts.append(f"- '{col}' 컬럼: 타입='{col_type}', 고유값={num_unique}개.")
        if pd.api.types.is_numeric_dtype(df[col]):
            summary_parts.append(f"  (최소={df[col].min():.2f}, 최대={df[col].max():.2f}, 평균={df[col].mean():.2f})")

    data_summary = "\n".join(summary_parts)

    # 시각화 라이브러리 및 요구사항 명시
    prompt = f"""
    아래는 사용자의 원래 질문, SQL 쿼리, 그리고 해당 쿼리 실행 결과의 JSON 데이터입니다.
    이 정보를 바탕으로 파이썬의 **Plotly Express** 라이브러리를 사용하여 가장 적합한 시각화 코드를 생성해주세요.
    ⚠️ 중요한 규칙: 아래 JSON 데이터의 수치를 절대 변경하지 마세요. 있는 그대로 사용하세요.
    **사용자의 원래 질문 (비즈니스 맥락):**
    "{original_question}"

    **SQL 쿼리:**
    ```sql
    {sql_query}
    ```

    **쿼리 결과 (JSON 데이터 예시 - 상위 5개 행만):**
    ```json
    {json.dumps(data[:5], indent=2, ensure_ascii=False)}
    ```

    **데이터 요약 통계:**
    {data_summary}

    **시각화 코드 생성 요구사항:**
    1.  데이터프레임을 로드하는 코드 (`df = pd.DataFrame(data)`)부터 시작해주세요. `data` 변수에는 전체 JSON 데이터가 리스트 형태로 들어있다고 가정합니다.
    2.  데이터의 컬럼 이름, 데이터 타입, 그리고 사용자의 원래 질문을 고려하여 **가장 적절하다고 판단되는 차트 유형** (예: 막대 그래프, 선 그래프, 파이 차트, 산점도 등)을 선택해주세요.
    3.  차트 유형에 맞게 **X축/Y축 또는 라벨/값에 가장 적절한 컬럼을 자동으로 선택**해주세요.
    4.  차트 제목은 사용자의 질문과 데이터 내용을 반영하여 적절하게 생성해주세요.
    5.  축 라벨도 컬럼의 의미에 맞게 적절하게 지정해주세요.
    6.  생성된 차트(`fig`)를 `fig.show()`로 **화면에 표시하는 코드와 함께**, `fig.to_json()`으로 JSON 문자열로 변환한 후, `json.loads` 및 `json.dumps(indent=2)`를 사용하여 예쁘게 만든 JSON을 '{original_question[:20].replace(' ', '_')}_chart.json' 파일명으로 저장하는 코드까지 포함해주세요.
    7.  파일 저장 후에는 "차트 JSON이 '[파일명]' 파일로 저장되었습니다." 메시지를 출력해주세요.
    8.  모든 코드를 하나의 파이썬 코드 블록(` ```python `)으로 감싸서 반환해주세요.
    9.  어떤 SQL 쿼리가 들어오든 **항상** 시각화 코드를 생성하려고 시도해주세요. 만약 시각화에 적합하지 않은 데이터라고 판단되면, 왜 그런지 간략히 주석으로 설명하고 간단한 테이블 출력 코드라도 생성해주세요.
    """
    return prompt


def suggest_chart_type(df: pd.DataFrame, question: str) -> str:
    scores = {
        "bar": 0,
        "line": 0,
        "pie": 0,
        "sankey": 0,
        "scatter": 0,
    }

    q = question.lower()
    if any(x in q for x in ["추이", "시간", "월별", "요일", "변화", "시계열", "타임라인"]):
        scores["line"] += 3
    if any(x in q for x in ["비교", "가장 많은", "상위", "top", "개수"]):
        scores["bar"] += 3
    if any(x in q for x in ["비율", "퍼센트", "구성", "분포", "점유율"]):
        scores["pie"] += 3
    if any(x in q for x in ["흐름", "전환", "이동", "경로"]):
        scores["sankey"] += 3
    if any(x in q for x in ["상관", "관계", "산점도"]):
        scores["scatter"] += 3

    num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
    cat_cols = [c for c in df.columns if pd.api.types.is_object_dtype(df[c]) or df[c].dtype.name == "category"]

    if len(cat_cols) == 1 and len(num_cols) == 1:
        scores["bar"] += 2
        scores["pie"] += 1
    if len(cat_cols) == 2 and len(num_cols) == 1:
        scores["sankey"] += 2
    if len(num_cols) >= 2:
        scores["scatter"] += 2
    if "date" in " ".join(df.columns).lower() or df.index.dtype.name.startswith("datetime"):
        scores["line"] += 2

    return sorted(scores.items(), key=lambda x: -x[1])[0][0]

# --- 시각화 코드 생성 및 실행 ---
if result_data_json:
    prompt_for_code_gen = get_visualization_prompt(state.question, state.sql, result_data_json)

    print("\n--- Gemini에게 동적 시각화 코드 생성 요청 중 ---")
    generated_code = llm_code_gen.generate(prompt_for_code_gen)

    print("\n--- Gemini가 생성한 시각화 코드 ---")
    print(generated_code)

    try:
        # 기존: Gemini가 생성한 시각화 코드 실행
        cleaned_code = generated_code.replace("```python", "").replace("```", "").strip()
        data = result_data_json
        import plotly.express as px
        import plotly.io as pio
        pio.templates.default = "plotly_white"
        exec(cleaned_code, globals(), locals())
        print("\n--- 생성된 시각화 코드 실행 완료 ---")

    except Exception as e:
        print(f"\n⚠️ 생성된 코드 실행 중 오류 발생: {e}")
        print("🔁 fallback 차트 자동 추천 및 실행")

        # ✅ fallback 시각화
        try:
            df = pd.DataFrame(result_data_json)

            if df.shape[1] < 2:
                print("⚠️ fallback 시각화를 위한 컬럼이 부족합니다.")
                display(df.head())
            else:
                chart_type = suggest_chart_type(df, state.question)
                print("🎯 추천된 fallback 차트 유형:", chart_type)

                import plotly.express as px
                import plotly.graph_objects as go

                fig = None
                if chart_type == "bar":
                    fig = px.bar(df, x=df.columns[0], y=df.columns[1])
                elif chart_type == "line":
                    fig = px.line(df, x=df.columns[0], y=df.columns[1])
                elif chart_type == "pie":
                    fig = px.pie(df, names=df.columns[0], values=df.columns[1])
                elif chart_type == "scatter":
                    fig = px.scatter(df, x=df.columns[0], y=df.columns[1])
                elif chart_type == "sankey":
                    print("⚠️ Sankey 차트는 자동 구성 미지원입니다. (고도화 필요)")

                if fig:
                    fig.show()
                    print("✅ fallback 차트 실행 완료")

        except Exception as ee:
            print(f"❌ fallback 시각화도 실패: {ee}")

else:
    print("\nJSON 데이터가 없으므로 시각화 코드 생성을 건너뛴니다.")

🔎 [selector] raw: {
  "tables": [
    "order_reviews",
    "orders",
    "order_items",
    "products"
  ],
  "rationale": "To calculate the monthly average review score per product category, we need 'review_score' from 'order_reviews', 'order_purchase_timestamp' for monthly grouping from 'orders', and 'product_category' from 'products'. These tables are linked via 'order_id' (order_reviews -> orders -> order_items) and 'product_id' (order_items -> products)."
}
🔎 [decomposer] raw: {
  "sub_questions": [],
  "sub_sqls": [],
  "final_sql": "WITH MonthlyCategoryReviews AS (\n    SELECT\n        TO_CHAR(ecommerce.orders.order_purchase_timestamp, 'YYYY-MM') AS sales_month,\n        ecommerce.products.product_category,\n        AVG(ecommerce.order_reviews.review_score) AS category_avg_review_score\n    FROM\n        ecommerce.orders\n    JOIN\n        ecommerce.order_reviews ON ecommerce.orders.order_id = ecommerce.order_reviews.order_id\n    JOIN\n        ecommerce.order_items ON ecommerce

### ✅ 결과 미리보기
이 보고서는 월별로 가장 높은 평균 리뷰 점수를 기록한 상품 카테고리를 보여줍니다. 각 월의 최고 카테고리 평균 리뷰 점수와 해당 월의 전체 상품 평균 리뷰 점수를 비교하여 확인할 수 있습니다.

| 월 | 최고 리뷰 카테고리 | 카테고리 월별 평균 리뷰 점수 | 전체 월별 평균 리뷰 점수 |
|:---|:-------------------|:-----------------------------|:-------------------------|

| sales_month   | top_product_category                    |   monthly_category_avg_review_score |   monthly_overall_avg_review_score |
|:--------------|:----------------------------------------|------------------------------------:|-----------------------------------:|
| 2023-06       | telephony                               |                                   1 |                               1    |
| 2023-07       | audio                                   |                                   5 |                               3.61 |
| 2023-07       | fashion_shoes                           |                                   5 |                               3.61 |
| 2023-07       | diapers_and_hygiene                     |                                   5 |                               3.61 |
| 2023-07       | food                                    |                                   5 |                               3.61 |
| 2023-07       | fashio_female_clothing                  |                                   5 |                               3.61 |
| 2023-10       | auto                                    |                                   5 |                               4.13 |
| 2023-10       | baby                                    |                                   5 |                               4.13 |
| 2023-10       | fashion_bags_accessories                |                                   5 |                               4.13 |
| 2023-10       | fixed_telephony                         |                                   5 |                               4.13 |
| 2023-10       | furniture_bedroom                       |                                   5 |                               4.13 |
| 2023-10       | health_beauty                           |                                   5 |                               4.13 |
| 2023-10       | music                                   |                                   5 |                               4.13 |
| 2023-10       | pet_shop                                |                                   5 |                               4.13 |
| 2023-10       | small_appliances                        |                                   5 |                               4.13 |
| 2023-10       | stationery                              |                                   5 |                               4.13 |
| 2023-11       | food                                    |                                   5 |                               4.09 |
| 2023-11       | kitchen_dining_laundry_garden_furniture |                                   5 |                               4.09 |
| 2023-11       | home_confort                            |                                   5 |                               4.09 |
| 2023-11       | fashion_shoes                           |                                   5 |                               4.09 |
| 2023-11       | costruction_tools_garden                |                                   5 |                               4.09 |
| 2023-12       | furniture_bedroom                       |                                   5 |                               3.99 |
| 2023-12       | music                                   |                                   5 |                               3.99 |
| 2023-12       | computers                               |                                   5 |                               3.99 |
| 2023-12       | kitchen_dining_laundry_garden_furniture |                                   5 |                               3.99 |
| 2023-12       | food_drink                              |                                   5 |                               3.99 |
| 2024-01       | costruction_tools_tools                 |                                   5 |                               4.08 |
| 2024-01       | home_comfort_2                          |                                   5 |                               4.08 |
| 2024-01       | food_drink                              |                                   5 |                               4.08 |
| 2024-01       | books_technical                         |                                   5 |                               4.08 |
| 2024-02       | books_technical                         |                                   5 |                               4.05 |
| 2024-02       | costruction_tools_tools                 |                                   5 |                               4.05 |
| 2024-02       | art                                     |                                   5 |                               4.05 |
| 2024-02       | cine_photo                              |                                   5 |                               4.05 |
| 2024-02       | christmas_supplies                      |                                   5 |                               4.05 |
| 2024-02       | signaling_and_security                  |                                   5 |                               4.05 |
| 2024-02       | agro_industry_and_commerce              |                                   5 |                               4.05 |
| 2024-02       | tablets_printing_image                  |                                   5 |                               4.05 |
| 2024-03       | industry_commerce_and_business          |                                   5 |                               4.17 |
| 2024-03       | la_cuisine                              |                                   5 |                               4.17 |
| 2024-03       | books_technical                         |                                   5 |                               4.17 |
| 2024-03       | tablets_printing_image                  |                                   5 |                               4.17 |
| 2024-03       | fashion_childrens_clothes               |                                   5 |                               4.17 |
| 2024-03       | fashion_sport                           |                                   5 |                               4.17 |
| 2024-03       | art                                     |                                   5 |                               4.17 |
| 2024-03       | home_comfort_2                          |                                   5 |                               4.17 |
| 2024-04       | food                                    |                                   5 |                               4.19 |
| 2024-04       | arts_and_craftmanship                   |                                   5 |                               4.19 |
| 2024-04       | small_appliances_home_oven_and_coffee   |                                   5 |                               4.19 |
| 2024-04       | books_imported                          |                                   5 |                               4.19 |
| 2024-05       | food_drink                              |                                   5 |                               4.19 |
| 2024-05       | agro_industry_and_commerce              |                                   5 |                               4.19 |
| 2024-05       | tablets_printing_image                  |                                   5 |                               4.19 |
| 2024-05       | home_construction                       |                                   5 |                               4.19 |
| 2024-05       | party_supplies                          |                                   5 |                               4.19 |
| 2024-05       | construction_tools_lights               |                                   5 |                               4.19 |
| 2024-05       | fashio_female_clothing                  |                                   5 |                               4.19 |
| 2024-05       | la_cuisine                              |                                   5 |                               4.19 |
| 2024-06       | agro_industry_and_commerce              |                                   5 |                               4.23 |
| 2024-06       | books_imported                          |                                   5 |                               4.23 |
| 2024-06       | books_technical                         |                                   5 |                               4.23 |
| 2024-06       | cds_dvds_musicals                       |                                   5 |                               4.23 |
| 2024-06       | construction_tools_lights               |                                   5 |                               4.23 |
| 2024-06       | costruction_tools_tools                 |                                   5 |                               4.23 |
| 2024-06       | furniture_bedroom                       |                                   5 |                               4.23 |
| 2024-06       | food                                    |                                   5 |                               4.23 |
| 2024-06       | fashion_sport                           |                                   5 |                               4.23 |
| 2024-07       | cine_photo                              |                                   5 |                               4.14 |
| 2024-07       | industry_commerce_and_business          |                                   5 |                               4.14 |
| 2024-07       | books_imported                          |                                   5 |                               4.14 |
| 2024-07       | books_technical                         |                                   5 |                               4.14 |
| 2024-07       | la_cuisine                              |                                   5 |                               4.14 |
| 2024-07       | fashion_childrens_clothes               |                                   5 |                               4.14 |
| 2024-07       | furniture_mattress_and_upholstery       |                                   5 |                               4.14 |
| 2024-08       | dvds_blu_ray                            |                                   5 |                               4.13 |
| 2024-08       | music                                   |                                   5 |                               4.13 |
| 2024-08       | cds_dvds_musicals                       |                                   5 |                               4.13 |
| 2024-08       | furniture_bedroom                       |                                   5 |                               4.13 |
| 2024-08       | flowers                                 |                                   5 |                               4.13 |
| 2024-08       | construction_tools_lights               |                                   5 |                               4.13 |
| 2024-08       | fashion_underwear_beach                 |                                   5 |                               4.13 |
| 2024-08       | costruction_tools_tools                 |                                   5 |                               4.13 |
| 2024-09       | fashion_sport                           |                                   5 |                               3.89 |
| 2024-09       | home_comfort_2                          |                                   5 |                               3.89 |
| 2024-09       | art                                     |                                   5 |                               3.89 |
| 2024-09       | cds_dvds_musicals                       |                                   5 |                               3.89 |
| 2024-10       | dvds_blu_ray                            |                                   5 |                               4.09 |
| 2024-10       | signaling_and_security                  |                                   5 |                               4.09 |
| 2024-10       | la_cuisine                              |                                   5 |                               4.09 |
| 2024-10       | cine_photo                              |                                   5 |                               4.09 |
| 2024-10       | flowers                                 |                                   5 |                               4.09 |
| 2024-11       | cine_photo                              |                                   5 |                               4    |
| 2024-11       | small_appliances_home_oven_and_coffee   |                                   5 |                               4    |
| 2024-11       | fashio_female_clothing                  |                                   5 |                               4    |
| 2024-11       | dvds_blu_ray                            |                                   5 |                               4    |
| 2024-12       | fashion_childrens_clothes               |                                   5 |                               3.66 |
| 2024-12       | small_appliances_home_oven_and_coffee   |                                   5 |                               3.66 |
| 2024-12       | music                                   |                                   5 |                               3.66 |
| 2024-12       | dvds_blu_ray                            |                                   5 |                               3.66 |
| 2024-12       | flowers                                 |                                   5 |                               3.66 |


--- 시각화를 위한 추출된 JSON 데이터 ---
[
  {
    "sales_month": "2023-06",
    "top_product_category": "telephony",
    "monthly_category_avg_review_score": 1.0,
    "monthly_overall_avg_review_score": 1.0
  },
  {
    "sales_month": "2023-07",
    "top_product_category": "audio",
    "monthly_category_avg_review_score": 5.0,
    "monthly_overall_avg_review_score": 3.61
  },
  {
    "sales_month": "2023-07",
    "top_product_category": "fashion_shoes",
    "monthly_category_avg_review_score": 5.0,
    "monthly_overall_avg_review_score": 3.61
  },
  {
    "sales_month": "2023-07",
    "top_product_category": "diapers_and_hygiene",
    "monthly_category_avg_review_score": 5.0,
    "monthly_overall_avg_review_score": 3.61
  },
  {
    "sales_month": "2023-07",
    "top_product_category": "food",
    "monthly_category_avg_review_score": 5.0,
    "monthly_overall_avg_review_score": 3.61
  },
  {
    "sales_month": "2023-07",
    "top_product_category": "fashio_female_clothing",
    "monthly_category

✅ fallback 차트 실행 완료


In [ ]:
import json
import pandas as pd
from IPython.display import Markdown, display, HTML
import os
import io

# --- 기존 LangGraph 에이전트 빌드 및 실행 부분 (생략 - 위에 있는 코드 그대로) ---
# ... (GEMINI_MODEL, engine, schema_text, app 등 정의 및 초기화) ...

# 예시 질문 (다양하게 바꿔서 테스트 가능)
# question = "20대와 30대 여성 고객이 구매한 제품 중 평균 구매 금액이 가장 높은 5개 영어 카테고리를 알려주세요."
# question = "주문 상태별 고객 수를 막대 그래프로 보여줘."
# question = "요일별 방문자 수 추이를 선 그래프로 보여줘."
question = "20대와 30대 여성 고객이 구매한 제품 중 평균 구매 금액이 가장 높은 5개 영어 카테고리를 알려주세요."
init = SQLState(
    question=question,
    schema_text=schema_text,
    schema_name=schema_name,
    schema_prefix=schema_prefix,
)

raw_result = app.invoke(init)
state = to_state(raw_result, init)

print("\n🪵 선택된 테이블:", state.selected_tables)
print("🪵 생성된 sub-questions:", state.sub_questions)
print("🪵 생성된 sub-sqls:", state.sub_sqls)
print("🪵 생성된 SQL:", state.sql)
print("🪵 에러 메시지:", state.error)

from IPython.display import Markdown, display
display(Markdown("### ✅ 결과 미리보기\n" + (state.result_preview or "_(출력 없음)_")))

result_data_json = None
if state.result_json:
    try:
        result_data_json = json.loads(state.result_json)
        print("\n--- 시각화를 위한 추출된 JSON 데이터 ---")
        print(json.dumps(result_data_json, indent=2, ensure_ascii=False))
    except json.JSONDecodeError as e:
        print(f"JSON 데이터 디코딩 오류: {e}")
        result_data_json = None
else:
    print("\n--- 시각화할 JSON 데이터 없음 ---")
    if state.error:
        print(f"에이전트 오류: {state.error}")
    elif state.result_preview == "| (결과 없음) |":
        print("SQL 쿼리 결과가 비어 있습니다.")

# --- Gemini 클라이언트 초기화 ---
llm_code_gen = GeminiClient(model=GEMINI_MODEL)


# --- 시각화 코드 생성을 위한 프롬프트 구성 함수 ---
def get_visualization_prompt(original_question: str, sql_query: str, data: list[dict]) -> str:
    if not data:
        return "데이터가 없어 시각화 코드를 생성할 수 없습니다."

    df = pd.DataFrame(data)

    # 데이터 요약 정보 생성
    summary_parts = []
    summary_parts.append(f"총 {len(df)}개의 행이 있습니다.")

    for col in df.columns:
        col_type = pd.api.types.infer_dtype(df[col])
        num_unique = df[col].nunique()
        summary_parts.append(f"- '{col}' 컬럼: 타입='{col_type}', 고유값={num_unique}개.")
        if pd.api.types.is_numeric_dtype(df[col]):
            summary_parts.append(f"  (최소={df[col].min():.2f}, 최대={df[col].max():.2f}, 평균={df[col].mean():.2f})")

    data_summary = "\n".join(summary_parts)

    # 시각화 라이브러리 및 요구사항 명시
    prompt = f"""
    아래는 사용자의 원래 질문, SQL 쿼리, 그리고 해당 쿼리 실행 결과의 JSON 데이터입니다.
    이 정보를 바탕으로 파이썬의 **Plotly Express** 라이브러리를 사용하여 가장 적합한 시각화 코드를 생성해주세요.

    **사용자의 원래 질문 (비즈니스 맥락):**
    "{original_question}"

    **SQL 쿼리:**
    ```sql
    {sql_query}
    ```

    **쿼리 결과 (JSON 데이터 예시 - 상위 5개 행만):**
    ```json
    {json.dumps(data[:5], indent=2, ensure_ascii=False)}
    ```

    **데이터 요약 통계:**
    {data_summary}

    **시각화 코드 생성 요구사항:**
    1.  데이터프레임을 로드하는 코드 (`df = pd.DataFrame(data)`)부터 시작해주세요. `data` 변수에는 전체 JSON 데이터가 리스트 형태로 들어있다고 가정합니다.
    2.  데이터의 컬럼 이름, 데이터 타입, 그리고 사용자의 원래 질문을 고려하여 **가장 적절하다고 판단되는 차트 유형** (예: 막대 그래프, 선 그래프, 파이 차트, 산점도 등)을 선택해주세요.
    3.  차트 유형에 맞게 **X축/Y축 또는 라벨/값에 가장 적절한 컬럼을 자동으로 선택**해주세요.
    4.  차트 제목은 사용자의 질문과 데이터 내용을 반영하여 적절하게 생성해주세요.
    5.  축 라벨도 컬럼의 의미에 맞게 적절하게 지정해주세요.
    6.  생성된 차트(`fig`)를 `fig.show()`로 **화면에 표시하는 코드와 함께**, `fig.to_json(indent=2)`를 사용하여 JSON 문자열로 변환하고, 이 JSON 문자열을 '{original_question[:20].replace(' ', '_')}_chart.json' 이라는 파일명으로 저장하는 코드까지 포함해주세요. (파일명은 질문의 앞 20글자를 사용하여 동적으로 생성)
    7.  파일 저장 후에는 "차트 JSON이 '[파일명]' 파일로 저장되었습니다." 메시지를 출력해주세요.
    8.  모든 코드를 하나의 파이썬 코드 블록(` ```python `)으로 감싸서 반환해주세요.
    9.  어떤 SQL 쿼리가 들어오든 **항상** 시각화 코드를 생성하려고 시도해주세요. 만약 시각화에 적합하지 않은 데이터라고 판단되면, 왜 그런지 간략히 주석으로 설명하고 간단한 테이블 출력 코드라도 생성해주세요.
    """
    return prompt

# --- 시각화 코드 생성 및 실행 ---
if result_data_json:
    prompt_for_code_gen = get_visualization_prompt(state.question, state.sql, result_data_json)

    print("\n--- Gemini에게 동적 시각화 코드 생성 요청 중 ---")
    generated_code = llm_code_gen.generate(prompt_for_code_gen)

    print("\n--- Gemini가 생성한 시각화 코드 ---")
    print(generated_code)

    try:
        # 코드 블록에서 Python 코드만 추출
        cleaned_code = generated_code.replace("```python", "").replace("```", "").strip()

        # 데이터 변수를 exec 환경에 주입
        data = result_data_json

        # Plotly Express를 사용하기 위한 임포트
        import plotly.express as px
        # Plotly에서 Jupyter/Colab 환경에서 차트를 잘 렌더링하도록 설정
        import plotly.io as pio
        pio.templates.default = "plotly_white"

        # 생성된 코드를 실행
        # locals()는 현재 로컬 스코프를 exec에 전달하여,
        # 생성된 코드 내에서 df, px 등의 변수를 사용할 수 있게 함
        exec(cleaned_code, globals(), locals())

        print("\n--- 생성된 시각화 코드 실행 완료 ---")

    except Exception as e:
        print(f"\n--- 생성된 코드 실행 중 오류 발생: {e} ---")
        print("생성된 코드를 직접 검토하고 수정해야 할 수 있습니다.")

else:
    print("\nJSON 데이터가 없으므로 시각화 코드 생성을 건너뛴니다.")

🔎 [selector] raw: {
  "tables": [
    "customer_info",
    "customers",
    "orders",
    "order_items",
    "products"
  ],
  "rationale": "To identify female customers in their 20s and 30s, 'customer_info' is needed for gender and age group. 'customers' links 'customer_info' to 'orders' via customer IDs. 'orders' provides the order details made by these customers. 'order_items' contains the price and product ID for each item within an order. Finally, 'products' is required to get the English product category for each product purchased."
}
🔎 [decomposer] raw: {
  "sub_questions": [
    "20대와 30대 여성 고객을 식별합니다.",
    "식별된 고객들의 구매 주문을 찾습니다.",
    "각 주문에 포함된 제품과 해당 제품의 구매 금액을 가져옵니다.",
    "각 제품의 카테고리를 확인합니다.",
    "각 카테고리별 평균 구매 금액을 계산합니다.",
    "평균 구매 금액이 가장 높은 상위 5개 카테고리를 정렬하여 반환합니다."
  ],
  "sub_sqls": [
    "SELECT customer_unique_id FROM ecommerce.customer_info WHERE customer_gender = 'female' AND customer_age_group IN ('20s', '30s');",
    "SELECT o.order_id, oi.product_id, oi.price

### ✅ 결과 미리보기
20대와 30대 여성 고객이 구매한 제품 중 평균 구매 금액이 가장 높은 상위 5개 영어 카테고리를 분석한 결과, 'computers' 카테고리가 평균 1118.3으로 가장 높은 구매 금액을 기록했습니다. 이어서 'agro_industry_and_commerce', 'small_appliances_home_oven_and_coffee' 등이 뒤를 이었습니다.

| product_category | average_purchase_amount |
|:-----------------|------------------------:|

| product_category                      |   average_purchase_amount |
|:--------------------------------------|--------------------------:|
| computers                             |                  1118.3   |
| agro_industry_and_commerce            |                   518.984 |
| small_appliances_home_oven_and_coffee |                   479.231 |
| home_appliances_2                     |                   303.611 |
| musical_instruments                   |                   218.471 |


--- 시각화를 위한 추출된 JSON 데이터 ---
[
  {
    "product_category": "computers",
    "average_purchase_amount": 1118.297
  },
  {
    "product_category": "agro_industry_and_commerce",
    "average_purchase_amount": 518.9843333333
  },
  {
    "product_category": "small_appliances_home_oven_and_coffee",
    "average_purchase_amount": 479.230625
  },
  {
    "product_category": "home_appliances_2",
    "average_purchase_amount": 303.6113333333
  },
  {
    "product_category": "musical_instruments",
    "average_purchase_amount": 218.4705263158
  }
]

--- Gemini에게 동적 시각화 코드 생성 요청 중 ---

--- Gemini가 생성한 시각화 코드 ---
```python
import pandas as pd
import plotly.express as px
import json

# 사용자의 원래 질문 (비즈니스 맥락)
original_question = "20대와 30대 여성 고객이 구매한 제품 중 평균 구매 금액이 가장 높은 5개 영어 카테고리를 알려주세요."

# 쿼리 결과 (JSON 데이터 예시)
data = [
  {
    "product_category": "computers",
    "average_purchase_amount": 1118.297
  },
  {
    "product_category": "agro_industry_and_commerce",
    "average_purchase_amount": 518.984


--- 생성된 코드 실행 중 오류 발생: write_json() got an unexpected keyword argument 'indent' ---
생성된 코드를 직접 검토하고 수정해야 할 수 있습니다.


In [ ]:
import json
import pandas as pd
from IPython.display import Markdown, display, HTML
import os
import io

# --- 기존 LangGraph 에이전트 빌드 및 실행 부분 (생략 - 위에 있는 코드 그대로) ---
# ... (GEMINI_MODEL, engine, schema_text, app 등 정의 및 초기화) ...

# 예시 질문 (다양하게 바꿔서 테스트 가능)
# question = "20대와 30대 여성 고객이 구매한 제품 중 평균 구매 금액이 가장 높은 5개 영어 카테고리를 알려주세요."
# question = "주문 상태별 고객 수를 막대 그래프로 보여줘."
# question = "요일별 방문자 수 추이를 선 그래프로 보여줘."
question = "20대와 30대 여성 고객이 구매한 제품 중 평균 구매 금액이 가장 높은 5개 영어 카테고리를 알려주세요."
init = SQLState(
    question=question,
    schema_text=schema_text,
    schema_name=schema_name,
    schema_prefix=schema_prefix,
)

raw_result = app.invoke(init)
state = to_state(raw_result, init)

print("\n🪵 선택된 테이블:", state.selected_tables)
print("🪵 생성된 sub-questions:", state.sub_questions)
print("🪵 생성된 sub-sqls:", state.sub_sqls)
print("🪵 생성된 SQL:", state.sql)
print("🪵 에러 메시지:", state.error)

from IPython.display import Markdown, display
display(Markdown("### ✅ 결과 미리보기\n" + (state.result_preview or "_(출력 없음)_")))

result_data_json = None
if state.result_json:
    try:
        result_data_json = json.loads(state.result_json)
        print("\n--- 시각화를 위한 추출된 JSON 데이터 ---")
        print(json.dumps(result_data_json, indent=2, ensure_ascii=False))
    except json.JSONDecodeError as e:
        print(f"JSON 데이터 디코딩 오류: {e}")
        result_data_json = None
else:
    print("\n--- 시각화할 JSON 데이터 없음 ---")
    if state.error:
        print(f"에이전트 오류: {state.error}")
    elif state.result_preview == "| (결과 없음) |":
        print("SQL 쿼리 결과가 비어 있습니다.")

# --- Gemini 클라이언트 초기화 ---
llm_code_gen = GeminiClient(model=GEMINI_MODEL)


# --- 시각화 코드 생성을 위한 프롬프트 구성 함수 ---
# 🚨🚨🚨 이 함수를 현재 코드의 해당 위치에 통째로 교체하세요! 🚨🚨🚨
def get_visualization_prompt(original_question: str, sql_query: str, data: list[dict]) -> str:
    if not data:
        return "데이터가 없어 시각화 코드를 생성할 수 없습니다."

    df = pd.DataFrame(data)

    # 데이터 요약 정보 생성
    summary_parts = []
    summary_parts.append(f"총 {len(df)}개의 행이 있습니다.")

    for col in df.columns:
        col_type = pd.api.types.infer_dtype(df[col])
        num_unique = df[col].nunique()
        summary_parts.append(f"- '{col}' 컬럼: 타입='{col_type}', 고유값={num_unique}개.")
        if pd.api.types.is_numeric_dtype(df[col]):
            summary_parts.append(f"  (최소={df[col].min():.2f}, 최대={df[col].max():.2f}, 평균={df[col].mean():.2f})")

    data_summary = "\n".join(summary_parts)

    # 시각화 라이브러리 및 요구사항 명시
    prompt = f"""
    아래는 사용자의 원래 질문, SQL 쿼리, 그리고 해당 쿼리 실행 결과의 JSON 데이터입니다.
    이 정보를 바탕으로 파이썬의 **Plotly Express** 라이브러리를 사용하여 가장 적합한 시각화 코드를 생성해주세요.

    **사용자의 원래 질문 (비즈니스 맥락):**
    "{original_question}"

    **SQL 쿼리:**
    ```sql
    {sql_query}
    ```

    **쿼리 결과 (JSON 데이터 예시 - 상위 5개 행만):**
    ```json
    {json.dumps(data[:5], indent=2, ensure_ascii=False)}
    ```

    **데이터 요약 통계:**
    {data_summary}

    **시각화 코드 생성 요구사항:**
    1.  데이터프레임을 로드하는 코드 (`df = pd.DataFrame(data)`)부터 시작해주세요. `data` 변수에는 전체 JSON 데이터가 리스트 형태로 들어있다고 가정합니다.
    2.  데이터의 컬럼 이름, 데이터 타입, 그리고 사용자의 원래 질문을 고려하여 **가장 적절하다고 판단되는 차트 유형** (예: 막대 그래프, 선 그래프, 파이 차트, 산점도 등)을 선택해주세요.
    3.  차트 유형에 맞게 **X축/Y축 또는 라벨/값에 가장 적절한 컬럼을 자동으로 선택**해주세요.
    4.  차트 제목은 사용자의 질문과 데이터 내용을 반영하여 적절하게 생성해주세요.
    5.  축 라벨도 컬럼의 의미에 맞게 적절하게 지정해주세요.
    6.  생성된 차트(`fig`)를 `fig.show()`로 **화면에 표시하는 코드와 함께**, `fig.to_json(indent=2)`를 사용하여 JSON 문자열로 변환하고, 이 JSON 문자열을 '{original_question[:20].replace(' ', '_')}_chart.json' 이라는 파일명으로 저장하는 코드까지 포함해주세요. (파일명은 질문의 앞 20글자를 사용하여 동적으로 생성)
    7.  파일 저장 후에는 "차트 JSON이 '[파일명]' 파일로 저장되었습니다." 메시지를 출력해주세요.
    8.  모든 코드를 하나의 파이썬 코드 블록(` ```python `)으로 감싸서 반환해주세요.
    9.  어떤 SQL 쿼리가 들어오든 **항상** 시각화 코드를 생성하려고 시도해주세요. 만약 시각화에 적합하지 않은 데이터라고 판단되면, 왜 그런지 간략히 주석으로 설명하고 간단한 테이블 출력 코드라도 생성해주세요.
    """
    return prompt
# --- 시각화 코드 생성 및 실행 ---
if result_data_json:
    prompt_for_code_gen = get_visualization_prompt(state.question, state.sql, result_data_json)

    print("\n--- Gemini에게 동적 시각화 코드 생성 요청 중 ---")
    generated_code = llm_code_gen.generate(prompt_for_code_gen)

    print("\n--- Gemini가 생성한 시각화 코드 ---")
    print(generated_code)

    try:
        # 코드 블록에서 Python 코드만 추출
        cleaned_code = generated_code.replace("```python", "").replace("```", "").strip()

        # 데이터 변수를 exec 환경에 주입
        data = result_data_json

        # Plotly Express를 사용하기 위한 임포트
        import plotly.express as px
        # Plotly에서 Jupyter/Colab 환경에서 차트를 잘 렌더링하도록 설정
        import plotly.io as pio
        pio.templates.default = "plotly_white"

        # 생성된 코드를 실행
        # locals()는 현재 로컬 스코프를 exec에 전달하여,
        # 생성된 코드 내에서 df, px 등의 변수를 사용할 수 있게 함
        exec(cleaned_code, globals(), locals())

        print("\n--- 생성된 시각화 코드 실행 완료 ---")

    except Exception as e:
        print(f"\n--- 생성된 코드 실행 중 오류 발생: {e} ---")
        print("생성된 코드를 직접 검토하고 수정해야 할 수 있습니다.")

else:
    print("\nJSON 데이터가 없으므로 시각화 코드 생성을 건너뛴니다.")

🔎 [selector] raw: {
  "tables": [
    "customer_info",
    "customers",
    "orders",
    "order_items",
    "products",
    "product_categories"
  ],
  "rationale": "To answer the question, we need to filter customers by age group (20s, 30s) and gender (female) from `customer_info`. Then, we link these customers to their orders using `customers` and `orders` tables. From `orders`, we get the `order_id` to access individual product purchases and their prices from `order_items`. Finally, we link `order_items` to `products` to get the `product_category` and then to `product_categories` to ensure we use the Englis
🔎 [decomposer] raw: {
  "sub_questions": [],
  "sub_sqls": [],
  "final_sql": "SELECT\n    pc.product_category,\n    AVG(oi.price * oi.quantity) AS average_purchase_amount\nFROM\n    ecommerce.customer_info AS ci\nJOIN\n    ecommerce.customers AS c ON ci.customer_unique_id = c.customer_unique_id\nJOIN\n    ecommerce.orders AS o ON c.customer_id = o.customer_id\nJOIN\n    ecommer

### ✅ 결과 미리보기
20대와 30대 여성 고객이 구매한 제품 중 평균 구매 금액이 가장 높은 상위 5개 영어 카테고리를 분석했습니다. 이 연령대 여성 고객들은 'computers' 카테고리에서 가장 높은 평균 구매 금액을 보였습니다.

| product_category | average_purchase_amount |
|:--------------------------------------|--------------------------:|

| product_category                      |   average_purchase_amount |
|:--------------------------------------|--------------------------:|
| computers                             |                  1118.3   |
| agro_industry_and_commerce            |                   518.984 |
| small_appliances_home_oven_and_coffee |                   479.231 |
| home_appliances_2                     |                   303.611 |
| musical_instruments                   |                   218.471 |


--- 시각화를 위한 추출된 JSON 데이터 ---
[
  {
    "product_category": "computers",
    "average_purchase_amount": 1118.297
  },
  {
    "product_category": "agro_industry_and_commerce",
    "average_purchase_amount": 518.9843333333
  },
  {
    "product_category": "small_appliances_home_oven_and_coffee",
    "average_purchase_amount": 479.230625
  },
  {
    "product_category": "home_appliances_2",
    "average_purchase_amount": 303.6113333333
  },
  {
    "product_category": "musical_instruments",
    "average_purchase_amount": 218.4705263158
  }
]

--- Gemini에게 동적 시각화 코드 생성 요청 중 ---

--- Gemini가 생성한 시각화 코드 ---
```python
import pandas as pd
import plotly.express as px
import json
import re

# 사용자의 원래 질문 (비즈니스 맥락)
original_question = "20대와 30대 여성 고객이 구매한 제품 중 평균 구매 금액이 가장 높은 5개 영어 카테고리를 알려주세요."

# 쿼리 결과 (JSON 데이터 예시 - 상위 5개 행만)
# 이 'data' 변수는 외부에서 주입된다고 가정합니다.
data = [
  {
    "product_category": "computers",
    "average_purchase_amount": 1118.297
  },
  {
    "product_category": "agro_industry


--- 생성된 코드 실행 중 오류 발생: to_json() got an unexpected keyword argument 'indent' ---
생성된 코드를 직접 검토하고 수정해야 할 수 있습니다.


In [ ]:
import json
import pandas as pd
from IPython.display import Markdown, display, HTML
import os
import io

# --- 기존 LangGraph 에이전트 빌드 및 실행 부분 (생략 - 위에 있는 코드 그대로) ---
# ... (GEMINI_MODEL, engine, schema_text, app 등 정의 및 초기화) ...

# 예시 질문 (다양하게 바꿔서 테스트 가능)
# question = "20대와 30대 여성 고객이 구매한 제품 중 평균 구매 금액이 가장 높은 5개 영어 카테고리를 알려주세요."
# question = "주문 상태별 고객 수를 막대 그래프로 보여줘."
# question = "요일별 방문자 수 추이를 선 그래프로 보여줘."
question = "20대와 30대 여성 고객이 구매한 제품 중 평균 구매 금액이 가장 높은 5개 영어 카테고리를 알려주세요."
init = SQLState(
    question=question,
    schema_text=schema_text,
    schema_name=schema_name,
    schema_prefix=schema_prefix,
)

raw_result = app.invoke(init)
state = to_state(raw_result, init)

print("\n🪵 선택된 테이블:", state.selected_tables)
print("🪵 생성된 sub-questions:", state.sub_questions)
print("🪵 생성된 sub-sqls:", state.sub_sqls)
print("🪵 생성된 SQL:", state.sql)
print("🪵 에러 메시지:", state.error)

from IPython.display import Markdown, display
display(Markdown("### ✅ 결과 미리보기\n" + (state.result_preview or "_(출력 없음)_")))

result_data_json = None
if state.result_json:
    try:
        result_data_json = json.loads(state.result_json)
        print("\n--- 시각화를 위한 추출된 JSON 데이터 ---")
        print(json.dumps(result_data_json, indent=2, ensure_ascii=False))
    except json.JSONDecodeError as e:
        print(f"JSON 데이터 디코딩 오류: {e}")
        result_data_json = None
else:
    print("\n--- 시각화할 JSON 데이터 없음 ---")
    if state.error:
        print(f"에이전트 오류: {state.error}")
    elif state.result_preview == "| (결과 없음) |":
        print("SQL 쿼리 결과가 비어 있습니다.")

# --- Gemini 클라이언트 초기화 ---
llm_code_gen = GeminiClient(model=GEMINI_MODEL)


# --- 시각화 코드 생성을 위한 프롬프트 구성 함수 ---
# 🚨🚨🚨 이 함수를 현재 코드의 해당 위치에 통째로 교체하세요! 🚨🚨🚨
def get_visualization_prompt(original_question: str, sql_query: str, data: list[dict]) -> str:
    if not data:
        return "데이터가 없어 시각화 코드를 생성할 수 없습니다."

    df = pd.DataFrame(data)

    # 데이터 요약 정보 생성
    summary_parts = []
    summary_parts.append(f"총 {len(df)}개의 행이 있습니다.")

    for col in df.columns:
        col_type = pd.api.types.infer_dtype(df[col])
        num_unique = df[col].nunique()
        summary_parts.append(f"- '{col}' 컬럼: 타입='{col_type}', 고유값={num_unique}개.")
        if pd.api.types.is_numeric_dtype(df[col]):
            summary_parts.append(f"  (최소={df[col].min():.2f}, 최대={df[col].max():.2f}, 평균={df[col].mean():.2f})")

    data_summary = "\n".join(summary_parts)

    # 시각화 라이브러리 및 요구사항 명시
    prompt = f"""
    아래는 사용자의 원래 질문, SQL 쿼리, 그리고 해당 쿼리 실행 결과의 JSON 데이터입니다.
    이 정보를 바탕으로 파이썬의 **Plotly Express** 라이브러리를 사용하여 가장 적합한 시각화 코드를 생성해주세요.

    **사용자의 원래 질문 (비즈니스 맥락):**
    "{original_question}"

    **SQL 쿼리:**
    ```sql
    {sql_query}
    ```

    **쿼리 결과 (JSON 데이터 예시 - 상위 5개 행만):**
    ```json
    {json.dumps(data[:5], indent=2, ensure_ascii=False)}
    ```

    **데이터 요약 통계:**
    {data_summary}

    **시각화 코드 생성 요구사항:**
    1.  데이터프레임을 로드하는 코드 (`df = pd.DataFrame(data)`)부터 시작해주세요. `data` 변수에는 전체 JSON 데이터가 리스트 형태로 들어있다고 가정합니다.
    2.  데이터의 컬럼 이름, 데이터 타입, 그리고 사용자의 원래 질문을 고려하여 **가장 적절하다고 판단되는 차트 유형** (예: 막대 그래프, 선 그래프, 파이 차트, 산점도 등)을 선택해주세요.
    3.  차트 유형에 맞게 **X축/Y축 또는 라벨/값에 가장 적절한 컬럼을 자동으로 선택**해주세요.
    4.  차트 제목은 사용자의 질문과 데이터 내용을 반영하여 적절하게 생성해주세요.
    5.  축 라벨도 컬럼의 의미에 맞게 적절하게 지정해주세요.
    6.  생성된 차트(`fig`)를 `fig.show()`로 **화면에 표시하는 코드와 함께**, `fig.to_json(indent=2)`를 사용하여 JSON 문자열로 변환하고, 이 JSON 문자열을 '{original_question[:20].replace(' ', '_')}_chart.json' 이라는 파일명으로 저장하는 코드까지 포함해주세요. (파일명은 질문의 앞 20글자를 사용하여 동적으로 생성)
    7.  파일 저장 후에는 "차트 JSON이 '[파일명]' 파일로 저장되었습니다." 메시지를 출력해주세요.
    8.  모든 코드를 하나의 파이썬 코드 블록(` ```python `)으로 감싸서 반환해주세요.
    9.  어떤 SQL 쿼리가 들어오든 **항상** 시각화 코드를 생성하려고 시도해주세요. 만약 시각화에 적합하지 않은 데이터라고 판단되면, 왜 그런지 간략히 주석으로 설명하고 간단한 테이블 출력 코드라도 생성해주세요.
    """
    return prompt
# --- 시각화 코드 생성 및 실행 ---
if result_data_json:
    prompt_for_code_gen = get_visualization_prompt(state.question, state.sql, result_data_json)

    print("\n--- Gemini에게 동적 시각화 코드 생성 요청 중 ---")
    generated_code = llm_code_gen.generate(prompt_for_code_gen)

    print("\n--- Gemini가 생성한 시각화 코드 ---")
    print(generated_code)

    try:
        # 코드 블록에서 Python 코드만 추출
        cleaned_code = generated_code.replace("```python", "").replace("```", "").strip()

        # 데이터 변수를 exec 환경에 주입
        data = result_data_json

        # Plotly Express를 사용하기 위한 임포트
        import plotly.express as px
        # Plotly에서 Jupyter/Colab 환경에서 차트를 잘 렌더링하도록 설정
        import plotly.io as pio
        pio.templates.default = "plotly_white"

        # 생성된 코드를 실행
        # locals()는 현재 로컬 스코프를 exec에 전달하여,
        # 생성된 코드 내에서 df, px 등의 변수를 사용할 수 있게 함
        exec(cleaned_code, globals(), locals())

        print("\n--- 생성된 시각화 코드 실행 완료 ---")

    except Exception as e:
        print(f"\n--- 생성된 코드 실행 중 오류 발생: {e} ---")
        print("생성된 코드를 직접 검토하고 수정해야 할 수 있습니다.")

else:
    print("\nJSON 데이터가 없으므로 시각화 코드 생성을 건너뛴니다.")

In [ ]:
import json
import pandas as pd
from IPython.display import Markdown, display, HTML
import os
import io

# --- 기존 LangGraph 에이전트 빌드 및 실행 부분 (생략 - 위에 있는 코드 그대로) ---
# ... (GEMINI_MODEL, engine, schema_text, app 등 정의 및 초기화) ...

# 예시 질문 (다양하게 바꿔서 테스트 가능)
# question = "20대와 30대 여성 고객이 구매한 제품 중 평균 구매 금액이 가장 높은 5개 영어 카테고리를 알려주세요."
# question = "주문 상태별 고객 수를 막대 그래프로 보여줘."
# question = "요일별 방문자 수 추이를 선 그래프로 보여줘."
question = "20대와 30대 여성 고객이 구매한 제품 중 평균 구매 금액이 가장 높은 5개 영어 카테고리를 알려주세요."
init = SQLState(
    question=question,
    schema_text=schema_text,
    schema_name=schema_name,
    schema_prefix=schema_prefix,
)

raw_result = app.invoke(init)
state = to_state(raw_result, init)

print("\n🪵 선택된 테이블:", state.selected_tables)
print("🪵 생성된 sub-questions:", state.sub_questions)
print("🪵 생성된 sub-sqls:", state.sub_sqls)
print("🪵 생성된 SQL:", state.sql)
print("🪵 에러 메시지:", state.error)

from IPython.display import Markdown, display
display(Markdown("### ✅ 결과 미리보기\n" + (state.result_preview or "_(출력 없음)_")))

result_data_json = None
if state.result_json:
    try:
        result_data_json = json.loads(state.result_json)
        print("\n--- 시각화를 위한 추출된 JSON 데이터 ---")
        print(json.dumps(result_data_json, indent=2, ensure_ascii=False))
    except json.JSONDecodeError as e:
        print(f"JSON 데이터 디코딩 오류: {e}")
        result_data_json = None
else:
    print("\n--- 시각화할 JSON 데이터 없음 ---")
    if state.error:
        print(f"에이전트 오류: {state.error}")
    elif state.result_preview == "| (결과 없음) |":
        print("SQL 쿼리 결과가 비어 있습니다.")

# --- Gemini 클라이언트 초기화 ---
llm_code_gen = GeminiClient(model=GEMINI_MODEL)


# --- 시각화 코드 생성을 위한 프롬프트 구성 함수 ---
def get_visualization_prompt(original_question: str, sql_query: str, data: list[dict]) -> str:
    if not data:
        return "데이터가 없어 시각화 코드를 생성할 수 없습니다."

    df = pd.DataFrame(data)

    # 데이터 요약 정보 생성
    summary_parts = []
    summary_parts.append(f"총 {len(df)}개의 행이 있습니다.")

    for col in df.columns:
        col_type = pd.api.types.infer_dtype(df[col])
        num_unique = df[col].nunique()
        summary_parts.append(f"- '{col}' 컬럼: 타입='{col_type}', 고유값={num_unique}개.")
        if pd.api.types.is_numeric_dtype(df[col]):
            summary_parts.append(f"  (최소={df[col].min():.2f}, 최대={df[col].max():.2f}, 평균={df[col].mean():.2f})")

    data_summary = "\n".join(summary_parts)

    # 시각화 라이브러리 및 요구사항 명시
    prompt = f"""
    아래는 사용자의 원래 질문, SQL 쿼리, 그리고 해당 쿼리 실행 결과의 JSON 데이터입니다.
    이 정보를 바탕으로 파이썬의 **Plotly Express** 라이브러리를 사용하여 가장 적합한 시각화 코드를 생성해주세요.

    **사용자의 원래 질문 (비즈니스 맥락):**
    "{original_question}"

    **SQL 쿼리:**
    ```sql
    {sql_query}
    ```

    **쿼리 결과 (JSON 데이터 예시 - 상위 5개 행만):**
    ```json
    {json.dumps(data[:5], indent=2, ensure_ascii=False)}
    ```

    **데이터 요약 통계:**
    {data_summary}

    **시각화 코드 생성 요구사항:**
    1.  데이터프레임을 로드하는 코드 (`df = pd.DataFrame(data)`)부터 시작해주세요. `data` 변수에는 전체 JSON 데이터가 리스트 형태로 들어있다고 가정합니다.
    2.  데이터의 컬럼 이름, 데이터 타입, 그리고 사용자의 원래 질문을 고려하여 **가장 적절하다고 판단되는 차트 유형** (예: 막대 그래프, 선 그래프, 파이 차트, 산점도 등)을 선택해주세요.
    3.  차트 유형에 맞게 **X축/Y축 또는 라벨/값에 가장 적절한 컬럼을 자동으로 선택**해주세요.
    4.  차트 제목은 사용자의 질문과 데이터 내용을 반영하여 적절하게 생성해주세요.
    5.  축 라벨도 컬럼의 의미에 맞게 적절하게 지정해주세요.
    6.  생성된 차트(`fig`)를 `fig.show()`로 **화면에 표시하는 코드와 함께**, `fig.to_json()`을 사용하여 JSON 문자열로 변환하고, 이 JSON 문자열을 '{original_question[:20].replace(' ', '_')}_chart.json' 이라는 파일명으로 저장하는 코드까지 포함해주세요. **(주의: `fig.to_json()` 호출 시 `indent` 인수를 사용하지 마세요. 호환성 문제로 오류가 발생할 수 있습니다.)** (파일명은 질문의 앞 20글자를 사용하여 동적으로 생성)
    7.  파일 저장 후에는 "차트 JSON이 '[파일명]' 파일로 저장되었습니다." 메시지를 출력해주세요.
    8.  모든 코드를 하나의 파이썬 코드 블록(` ```python `)으로 감싸서 반환해주세요.
    9.  어떤 SQL 쿼리가 들어오든 **항상** 시각화 코드를 생성하려고 시도해주세요. 만약 시각화에 적합하지 않은 데이터라고 판단되면, 왜 그런지 간략히 주석으로 설명하고 간단한 테이블 출력 코드라도 생성해주세요.
    """
    return prompt

# --- 시각화 코드 생성 및 실행 ---
if result_data_json:
    prompt_for_code_gen = get_visualization_prompt(state.question, state.sql, result_data_json)

    print("\n--- Gemini에게 동적 시각화 코드 생성 요청 중 ---")
    generated_code = llm_code_gen.generate(prompt_for_code_gen)

    print("\n--- Gemini가 생성한 시각화 코드 ---")
    print(generated_code)

    try:
        # 코드 블록에서 Python 코드만 추출
        cleaned_code = generated_code.replace("```python", "").replace("```", "").strip()

        # 데이터 변수를 exec 환경에 주입
        data = result_data_json

        # Plotly Express를 사용하기 위한 임포트
        import plotly.express as px
        # Plotly에서 Jupyter/Colab 환경에서 차트를 잘 렌더링하도록 설정
        import plotly.io as pio
        pio.templates.default = "plotly_white"

        # 생성된 코드를 실행
        # locals()는 현재 로컬 스코프를 exec에 전달하여,
        # 생성된 코드 내에서 df, px 등의 변수를 사용할 수 있게 함
        exec(cleaned_code, globals(), locals())

        print("\n--- 생성된 시각화 코드 실행 완료 ---")

    except Exception as e:
        print(f"\n--- 생성된 코드 실행 중 오류 발생: {e} ---")
        print("생성된 코드를 직접 검토하고 수정해야 할 수 있습니다.")

else:
    print("\nJSON 데이터가 없으므로 시각화 코드 생성을 건너뛴니다.")

🔎 [selector] raw: {
  "tables": [
    "customer_info",
    "customers",
    "orders",
    "order_items",
    "products",
    "product_categories"
  ],
  "rationale": "To identify female customers in their 20s and 30s, 'customer_info' is needed for 'customer_gender' and 'customer_age_group'. 'customers' links 'customer_unique_id' from 'customer_info' to 'customer_id' used in 'orders'. 'orders' provides the 'order_id' and 'customer_id' for purchases. 'order_items' contains the 'price' and 'product_id' for each item within an order, which is necessary to calculate the average purchase amount. 'products' links 'pro
🔎 [decomposer] raw: {
  "sub_questions": [
    "20대와 30대 여성 고객을 식별하는 방법은 무엇인가요?",
    "식별된 고객들이 구매한 제품의 주문 정보와 해당 제품의 카테고리를 어떻게 연결할 수 있나요?",
    "각 제품 구매 건별 구매 금액(가격 * 수량)을 어떻게 계산할 수 있나요?",
    "각 제품 카테고리별 평균 구매 금액을 어떻게 계산하고, 이를 기준으로 정렬할 수 있나요?",
    "평균 구매 금액이 가장 높은 상위 5개 카테고리를 어떻게 선택할 수 있나요?"
  ],
  "sub_sqls": [
    "SELECT customer_unique_id FROM ecommerce.customer_info WHER

### ✅ 결과 미리보기
20대와 30대 여성 고객이 구매한 제품 중 평균 구매 금액이 가장 높은 상위 5개 영어 카테고리는 컴퓨터, 농업/산업/상업, 소형 가전/오븐/커피, 가전제품 2, 악기 순입니다. 특히 컴퓨터 카테고리의 평균 구매 금액이 다른 카테고리에 비해 월등히 높게 나타났습니다.

| product_category | average_purchase_amount |
|:--------------------------------------|--------------------------:|

| product_category                      |   average_purchase_amount |
|:--------------------------------------|--------------------------:|
| computers                             |                  1118.3   |
| agro_industry_and_commerce            |                   518.984 |
| small_appliances_home_oven_and_coffee |                   479.231 |
| home_appliances_2                     |                   303.611 |
| musical_instruments                   |                   218.471 |


--- 시각화를 위한 추출된 JSON 데이터 ---
[
  {
    "product_category": "computers",
    "average_purchase_amount": 1118.297
  },
  {
    "product_category": "agro_industry_and_commerce",
    "average_purchase_amount": 518.9843333333
  },
  {
    "product_category": "small_appliances_home_oven_and_coffee",
    "average_purchase_amount": 479.230625
  },
  {
    "product_category": "home_appliances_2",
    "average_purchase_amount": 303.6113333333
  },
  {
    "product_category": "musical_instruments",
    "average_purchase_amount": 218.4705263158
  }
]

--- Gemini에게 동적 시각화 코드 생성 요청 중 ---

--- Gemini가 생성한 시각화 코드 ---
```python
import pandas as pd
import plotly.express as px
import json

# 쿼리 결과 (JSON 데이터 예시)
data = [
  {
    "product_category": "computers",
    "average_purchase_amount": 1118.297
  },
  {
    "product_category": "agro_industry_and_commerce",
    "average_purchase_amount": 518.9843333333
  },
  {
    "product_category": "small_appliances_home_oven_and_coffee",
    "average_purchase_am

차트 JSON이 '20대와_30대_여성_고객이_구매한__chart.json' 파일로 저장되었습니다.

--- 생성된 시각화 코드 실행 완료 ---


In [ ]:
# 예시 질문
question = "20대와 30대 여성 고객이 구매한 제품 중 평균 구매 금액이 가장 높은 5개 영어 카테고리를 알려주세요."

init = SQLState(
    question=question,
    schema_text=schema_text,
    schema_name=schema_name,
    schema_prefix=schema_prefix,
)

raw_result = app.invoke(init)
state = to_state(raw_result, init)

print("\n🪵 선택된 테이블:", state.selected_tables)
print("🪵 생성된 sub-questions:", state.sub_questions)
print("🪵 생성된 sub-sqls:", state.sub_sqls)
print("🪵 생성된 SQL:", state.sql)
print("🪵 에러 메시지:", state.error)

from IPython.display import Markdown, display
display(Markdown("### ✅ 결과 미리보기\n" + (state.result_preview or "_(출력 없음)_")))

🔎 [selector] raw: {"tables": ["customers", "customer_info", "orders", "order_items", "products", "product_categories"], "rationale": "To identify the top 5 English product categories with the highest average purchase amount by female customers in their 20s and 30s, we need to join data across multiple tables.  `customer_info` table provides customer demographics (age and gender). `customers` table links unique customer IDs to order IDs. `orders` table contains order information. `order_items` table stores product details within each order, including price and quantity. `products` table links product IDs to prod
🔎 [decomposer] raw: {"sub_questions": ["20대 여성과 30대 여성 고객을 각각 식별합니다.", "각 고객이 구매한 제품의 주문 정보를 가져옵니다.", "각 제품의 카테고리 정보를 가져옵니다.", "20대 여성과 30대 여성 고객이 구매한 제품의 평균 구매 금액을 계산합니다.", "평균 구매 금액이 가장 높은 상위 5개 카테고리를 선택합니다."], "sub_sqls": ["SELECT customer_unique_id FROM ecommerce.customer_info WHERE customer_age_group IN ('20s', '30s') AND customer_gender = 'female';", "SELECT oi.product_id,

### ✅ 결과 미리보기
20대와 30대 여성 고객의 평균 구매 금액이 가장 높은 상위 5개 제품 카테고리를 분석했습니다. 컴퓨터, 소형 주방 가전, 농업 및 상업, 가전 제품, 악기 순으로 평균 구매 금액이 높았습니다.

| Product Category |
|---|---|



| product_category                      |
|:--------------------------------------|
| computers                             |
| small_appliances_home_oven_and_coffee |
| agro_industry_and_commerce            |
| home_appliances_2                     |
| musical_instruments                   |

In [ ]:
# 예시 질문
question = "20대와 30대 여성 고객이 구매한 제품 중 평균 구매 금액이 가장 높은 5개 영어 카테고리를 알려주세요."

init = SQLState(
    question=question,
    schema_text=schema_text,
    schema_name=schema_name,
    schema_prefix=schema_prefix,
)

raw_result = app.invoke(init)
state = to_state(raw_result, init)

print("\n🪵 선택된 테이블:", state.selected_tables)
print("🪵 생성된 sub-questions:", state.sub_questions)
print("🪵 생성된 sub-sqls:", state.sub_sqls)
print("🪵 생성된 SQL:", state.sql)
print("🪵 에러 메시지:", state.error)

from IPython.display import Markdown, display
display(Markdown("### ✅ 결과 미리보기\n" + (state.result_preview or "_(출력 없음)_")))

🔎 [selector] raw: {
  "tables": [
    "customer_info",
    "customers",
    "orders",
    "order_items",
    "products"
  ],
  "rationale": "To answer the question, we need customer demographic information (age group and gender) from `customer_info`. This needs to be linked to `customers` to get the `customer_id` used in `orders`. From `orders`, we get the `order_id` for purchases. `order_items` provides the `product_id` and `price` for each item in an order. Finally, `products` is needed to get the `product_category` (which is the English category name) for each `product_id`. These tables allow filtering custo
🔎 [decomposer] raw: {
  "sub_questions": [
    "20대와 30대 여성 고객을 식별하는 방법은 무엇인가요?",
    "식별된 고객들이 구매한 제품과 해당 제품의 카테고리를 어떻게 연결할 수 있나요?",
    "각 제품 구매 건에 대한 총 구매 금액(단가 * 수량)을 어떻게 계산하나요?",
    "각 영어 카테고리별로 제품의 평균 구매 금액을 어떻게 집계하나요?",
    "평균 구매 금액이 가장 높은 상위 5개 카테고리를 어떻게 정렬하고 선택하나요?"
  ],
  "sub_sqls": [
    "SELECT customer_unique_id FROM ecommerce.customer_info WHERE customer_gender 

### ✅ 결과 미리보기
20대와 30대 여성 고객이 구매한 제품 중 평균 구매 금액이 가장 높은 상위 5개 영어 카테고리는 컴퓨터, 농업/산업/상업, 소형 가전/오븐/커피, 가전제품2, 악기입니다. 특히 컴퓨터 카테고리의 평균 구매 금액이 가장 높게 나타났습니다.

| 제품 카테고리 | 평균 구매 금액 |
|---|---|

| product_category                      |   average_purchase_amount |
|:--------------------------------------|--------------------------:|
| computers                             |                  1118.3   |
| agro_industry_and_commerce            |                   518.984 |
| small_appliances_home_oven_and_coffee |                   479.231 |
| home_appliances_2                     |                   303.611 |
| musical_instruments                   |                   218.471 |

In [ ]:
# 예시 질문
question = "각 연령대별로 가장 많이 판매된 상품 카테고리는 무엇이며, 해당 카테고리의 총 매출액과 판매된 상품의 평균 가격은 얼마인가요? (결과는 연령대와 매출액이 높은 순서로 정렬해주세요)"

init = SQLState(
    question=question,
    schema_text=schema_text,
    schema_name=schema_name,
    schema_prefix=schema_prefix,
)

raw_result = app.invoke(init)
state = to_state(raw_result, init)

print("\n🪵 선택된 테이블:", state.selected_tables)
print("🪵 생성된 sub-questions:", state.sub_questions)
print("🪵 생성된 sub-sqls:", state.sub_sqls)
print("🪵 생성된 SQL:", state.sql)
print("🪵 에러 메시지:", state.error)

from IPython.display import Markdown, display
display(Markdown("### ✅ 결과 미리보기\n" + (state.result_preview or "_(출력 없음)_")))

🔎 [selector] raw: {
  "tables": [
    "customer_info",
    "customers",
    "orders",
    "order_items",
    "products"
  ],
  "rationale": "To determine the most sold product category, total sales, and average price per age group, we need to link customer age information to product sales data. 'customer_info' provides the 'customer_age_group'. This links to 'customers' via 'customer_unique_id'. 'customers' links to 'orders' via 'customer_id'. 'orders' links to 'order_items' via 'order_id', which contains the 'price' and 'quantity' for sales calculations and 'product_id'. Finally, 'order_items' links to 'produc
🔎 [decomposer] raw: {
  "sub_questions": [
    "각 연령대별로 상품 카테고리별 총 매출액과 판매된 상품의 평균 가격을 계산합니다.",
    "각 연령대 내에서 총 매출액이 가장 높은 상품 카테고리를 식별합니다.",
    "최종 결과를 연령대와 총 매출액이 높은 순서로 정렬합니다."
  ],
  "sub_sqls": [
    "WITH CategorySales AS (SELECT eci.customer_age_group, ep.product_category, SUM(eoi.price * eoi.quantity) AS total_revenue, SUM(eoi.price * eoi.quantity) / SUM(eoi.quantity) A

### ✅ 결과 미리보기
각 연령대별로 가장 높은 매출을 기록한 상품 카테고리를 분석한 결과입니다. 각 연령대에서 가장 많이 판매된 카테고리의 총 매출액과 해당 상품의 평균 가격을 확인할 수 있습니다.

| 연령대 | 상품 카테고리 | 총 매출액 | 평균 상품 가격 |
|:---|:---|:---|:---|

| customer_age_group   | product_category   |   total_revenue |   average_product_price |
|:---------------------|:-------------------|----------------:|------------------------:|
| 10s                  | bed_bath_table     |         81639.4 |                 44.5873 |
| 20s                  | bed_bath_table     |        197448   |                 68.1795 |
| 30s                  | health_beauty      |        382312   |                147.839  |
| 40s                  | watches_gifts      |        666964   |                303.856  |

In [ ]:
# 예시 질문 1.5pro
question = "각 연령대별로 가장 많이 판매된 상품 카테고리는 무엇이며, 해당 카테고리의 총 매출액과 판매된 상품의 평균 가격은 얼마인가요? (결과는 연령대와 매출액이 높은 순서로 정렬해주세요)"

init = SQLState(
    question=question,
    schema_text=schema_text,
    schema_name=schema_name,
    schema_prefix=schema_prefix,
)

raw_result = app.invoke(init)
state = to_state(raw_result, init)

print("\n🪵 선택된 테이블:", state.selected_tables)
print("🪵 생성된 sub-questions:", state.sub_questions)
print("🪵 생성된 sub-sqls:", state.sub_sqls)
print("🪵 생성된 SQL:", state.sql)
print("🪵 에러 메시지:", state.error)

from IPython.display import Markdown, display
display(Markdown("### ✅ 결과 미리보기\n" + (state.result_preview or "_(출력 없음)_")))

🔎 [selector] raw: {"tables": ["customers", "customer_info", "orders", "order_items", "products", "product_categories"], "rationale": "To answer the question about the best-selling product category by age group, along with total sales and average price, we need to join data from multiple tables.\n\n1. **customer_info**: Provides the `customer_age_group`.\n2. **customers**: Links `customer_unique_id` to `customer_id` which is used in the `orders` table.\n3. **orders**: Connects customers to their orders and provides the necessary link to order items.\n4. **order_items**: Contains `product_id`, `quantity`, and `pr
🔎 [decomposer] raw: {"sub_questions": ["1. 각 연령대별로 판매된 상품들의 카테고리를 확인합니다.", "2. 각 연령대별로 가장 많이 판매된 카테고리를 찾습니다.", "3. 각 연령대별 최다 판매 카테고리의 총 매출액과 평균 가격을 계산합니다.", "4. 결과를 연령대와 매출액이 높은 순서로 정렬합니다."], "sub_sqls": ["SELECT ci.customer_age_group, p.product_category, COUNT(*) AS category_count FROM ecommerce.orders o JOIN ecommerce.customers c ON o.customer_id = c.customer_id JOIN ecommerce

### ✅ 결과 미리보기
각 연령대별로 가장 많이 팔린 상품 카테고리와 그에 따른 총 매출액, 평균 가격을 분석했습니다.  10대는 'telephony', 20대부터 40대까지는 'bed_bath_table' 카테고리의 상품이 가장 많이 판매된 것을 확인할 수 있습니다.

| 연령대 | 카테고리 | 총 매출액 | 평균 가격 |
|---|---|---|---|


| customer_age_group   | product_category   |   total_revenue |   avg_price |
|:---------------------|:-------------------|----------------:|------------:|
| 10s                  | telephony          |         59657.6 |     27.454  |
| 20s                  | bed_bath_table     |        195025   |     69.4779 |
| 30s                  | bed_bath_table     |        320812   |    106.124  |
| 40s                  | bed_bath_table     |        365227   |    145.974  |

In [ ]:
# 예시 질문
question = "20대 남성 고객 중 'promotion' 캠페인을 받고 구매한 비율은?"

init = SQLState(
    question=question,
    schema_text=schema_text,
    schema_name=schema_name,
    schema_prefix=schema_prefix,
)

raw_result = app.invoke(init)
state = to_state(raw_result, init)

print("\n🪵 선택된 테이블:", state.selected_tables)
print("🪵 생성된 sub-questions:", state.sub_questions)
print("🪵 생성된 sub-sqls:", state.sub_sqls)
print("🪵 생성된 SQL:", state.sql)
print("🪵 에러 메시지:", state.error)

from IPython.display import Markdown, display
display(Markdown("### ✅ 결과 미리보기\n" + (state.result_preview or "_(출력 없음)_")))

🔎 [selector] raw: {"tables": ["customers", "customer_info", "orders", "marketing_campaign"], "rationale": "To calculate the purchase rate of male customers in their 20s who received the 'promotion' campaign, we need to identify 20s male customers (customer_info, customers), check if they made a purchase (orders), and if they received the 'promotion' campaign (marketing_campaign)."}
🔎 [decomposer] raw: {"sub_questions": ["20대 남성 고객의 수는?", "'promotion' 캠페인을 받고 구매한 20대 남성 고객의 수는?"], "sub_sqls": ["SELECT COUNT(DISTINCT ci.customer_unique_id) FROM ecommerce.customer_info AS ci WHERE ci.customer_age_group = '20s' AND ci.customer_gender = 'male';", "SELECT COUNT(DISTINCT ci.customer_unique_id) FROM ecommerce.customer_info AS ci INNER JOIN ecommerce.customers AS c ON ci.customer_unique_id = c.customer_unique_id INNER JOIN ecommerce.orders AS o ON c.customer_id = o.customer_id INNER JOIN ecommerce.marketing_campaign AS mc ON o.order_purchase_timestamp BETWEEN mc.send_timestamp AND mc.send_times

### ✅ 결과 미리보기
20대 남성 고객 중 'promotion' 캠페인을 통해 구매를 완료한 비율은 48.22% 입니다.  해당 결과는 이커머스 데이터를 기반으로 'promotion' 캠페인 발송 후 1일 이내 구매 건을 분석하여 도출되었습니다.

| Promotion Conversion Rate |
|-------------------------|


|   ?column? |
|-----------:|
|    48.2241 |

In [ ]:
# 예시 질문
question = "20대 여성 중 쿠폰을 발급받고 캠페인 메시지도 받은 후 구매까지 완료한 고객 수는?"

init = SQLState(
    question=question,
    schema_text=schema_text,
    schema_name=schema_name,
    schema_prefix=schema_prefix,
)

raw_result = app.invoke(init)
state = to_state(raw_result, init)

print("\n🪵 선택된 테이블:", state.selected_tables)
print("🪵 생성된 sub-questions:", state.sub_questions)
print("🪵 생성된 sub-sqls:", state.sub_sqls)
print("🪵 생성된 SQL:", state.sql)
print("🪵 에러 메시지:", state.error)

from IPython.display import Markdown, display
display(Markdown("### ✅ 결과 미리보기\n" + (state.result_preview or "_(출력 없음)_")))

🔎 [selector] raw: {"tables": ["customer_info", "customers", "orders", "coupon", "marketing_campaign"], "rationale": "20대 여성, 쿠폰 발급, 캠페인 메시지 수신, 구매 완료 조건을 만족하는 고객 수를 계산하려면 다음 테이블이 필요합니다.\n\n* customer_info: 고객의 성별과 연령대 정보를 얻기 위해 필요합니다. 'customer_age_group'과 'customer_gender'를 사용하여 20대 여성 고객을 식별합니다.\n* customers: 고객의 고유 식별자를 기반으로 주문 정보와 연결하기 위해 필요합니다.\n* orders: 구매 완료 여부를 확인하기 위해 필요합니다. 'order_purchase_timestamp'가 존재하는 경우 구매가 완료된 것으로 간주합니다.\n* coupon: 쿠폰 발급 여부를 확인하기 위해 필요합니다.\n* marketing_campaign: 캠페인 메시지 수신 여부를 확인하기 위해 필요합니다."}
🔎 [decomposer] raw: {"sub_questions": ["20대 여성 고객을 식별합니다.", "쿠폰을 발급받은 고객을 식별합니다.", "캠페인 메시지를 받은 고객을 식별합니다.", "구매를 완료한 고객을 식별합니다."], "sub_sqls": ["SELECT customer_unique_id FROM ecommerce.customer_info WHERE customer_age_group = '20s' AND customer_gender = 'female';", "SELECT DISTINCT customer_unique_id FROM ecommerce.coupon AS c LEFT JOIN ecommerce.promotion_coupon_mapping AS pcm ON c.coupon_id = pcm.coupon_id;", "SELECT DISTINCT customer_unique_id FROM ecommerc

### ✅ 결과 미리보기
20대 여성 중 쿠폰을 발급받고 캠페인 메시지를 받은 후 구매까지 완료한 고객은 총 14,830명입니다.  해당 쿼리는 고객 정보, 주문 내역, 마케팅 캠페인 데이터를 조합하여 조건에 맞는 고객 수를 계산했습니다.

| count |
|-------|


|   count |
|--------:|
|   14830 |

In [ ]:
# 예시 질문
question = "각각 연령대별로 어떤 카테고리의 매출이 가장 높았는지 알려줘"

init = SQLState(
    question=question,
    schema_text=schema_text,
    schema_name=schema_name,
    schema_prefix=schema_prefix,
)

raw_result = app.invoke(init, config={"recursion_limit": 70})

state = to_state(raw_result, init)

print("\n🪵 선택된 테이블:", state.selected_tables)
print("🪵 생성된 sub-questions:", state.sub_questions)
print("🪵 생성된 sub-sqls:", state.sub_sqls)
print("🪵 생성된 SQL:", state.sql)
print("🪵 에러 메시지:", state.error)

from IPython.display import Markdown, display
display(Markdown("### ✅ 결과 미리보기\n" + (state.result_preview or "_(출력 없음)_")))

🔎 [selector] raw: {
  "tables": [
    "customer_info",
    "customers",
    "orders",
    "order_items",
    "products"
  ],
  "rationale": "To determine the highest-selling product category by age group, we need customer age information from 'customer_info', which links to 'customers'. 'customers' links to 'orders' to get order details. 'orders' links to 'order_items' to get product quantities and prices for sales calculation. Finally, 'order_items' links to 'products' to identify the product category for each sale."
}
🔎 [decomposer] raw: {
  "sub_questions": [
    "각 연령대별로 상품 카테고리별 총 매출을 계산하려면 어떤 테이블을 조인해야 하는가?",
    "고객의 연령대 정보는 어떤 테이블에 있으며, 이를 매출 데이터와 연결하기 위한 조인 경로는 무엇인가?",
    "각 연령대별로 가장 높은 매출을 기록한 카테고리를 식별하기 위해 어떤 SQL 함수나 기법을 사용해야 하는가?"
  ],
  "sub_sqls": [
    "SELECT ci.customer_age_group, p.product_category, SUM(oi.price * oi.quantity) AS total_sales FROM ecommerce.order_items AS oi JOIN ecommerce.products AS p ON oi.product_id = p.product_id JOIN ecommerce.orders AS o ON oi.

### ✅ 결과 미리보기
각 연령대별로 가장 높은 매출을 기록한 제품 카테고리를 분석한 결과입니다. 10대와 20대에서는 '침구/욕실용품'이, 30대에서는 '건강/미용'이, 40대에서는 '시계/선물용품'이 가장 높은 매출을 보였습니다.

| customer_age_group | product_category | total_sales |

| customer_age_group   | product_category   |   total_sales |
|:---------------------|:-------------------|--------------:|
| 10s                  | bed_bath_table     |       83263.7 |
| 20s                  | bed_bath_table     |      200469   |
| 30s                  | health_beauty      |      391161   |
| 40s                  | watches_gifts      |      684873   |

In [ ]:
# 예시 질문
question = "리뷰 점수 3 미만 비율이 20% 이상인 카테고리의 월별 매출 추이"

init = SQLState(
    question=question,
    schema_text=schema_text,
    schema_name=schema_name,
    schema_prefix=schema_prefix,
)

raw_result = app.invoke(init)
state = to_state(raw_result, init)

print("\n🪵 선택된 테이블:", state.selected_tables)
print("🪵 생성된 sub-questions:", state.sub_questions)
print("🪵 생성된 sub-sqls:", state.sub_sqls)
print("🪵 생성된 SQL:", state.sql)
print("🪵 에러 메시지:", state.error)

from IPython.display import Markdown, display
display(Markdown("### ✅ 결과 미리보기\n" + (state.result_preview or "_(출력 없음)_")))

🔎 [selector] raw: {
  "tables": [
    "order_reviews",
    "orders",
    "order_items",
    "products"
  ],
  "rationale": "To determine the ratio of review scores below 3 per category, 'order_reviews' (for review_score and order_id) and 'products' (for product_category_name and product_id) are needed. These tables are linked via 'orders' (for order_id) and 'order_items' (for order_id and product_id). To calculate monthly sales trends, 'order_items' (for price) and 'orders' (for order_purchase_timestamp to extract month) are required. All these tables are interconnected to fulfill the requirements of filtering 
🔎 [decomposer] raw: {
  "sub_questions": [
    "Identify product categories where the proportion of reviews with a score less than 3 is 20% or more.",
    "Calculate the monthly sales trend for the identified product categories."
  ],
  "sub_sqls": [
    "SELECT p.product_category_name FROM ecommerce.order_reviews orv JOIN ecommerce.orders o ON orv.order_id = o.order_id JOIN eco

### ✅ 결과 미리보기
리뷰 점수 3점 미만 비율이 20% 이상인 카테고리들의 월별 매출 추이를 보여주는 데이터입니다. 각 월별로 해당 카테고리들의 총 매출액을 확인할 수 있습니다.

| sales_month | product_category_name | monthly_sales |
|---|---|---|

| sales_month   | product_category_name            |   monthly_sales |
|:--------------|:---------------------------------|----------------:|
| 2016-10       |                                  |           65.89 |
| 2016-10       | audio                            |          156.99 |
| 2016-10       | fashion_roupa_masculina          |           24.9  |
| 2016-10       | fraldas_higiene                  |          134.9  |
| 2016-10       | moveis_escritorio                |         1300.88 |
| 2016-10       | telefonia_fixa                   |          704.88 |
| 2017-01       |                                  |         1702.64 |
| 2017-01       | casa_conforto_2                  |           72.8  |
| 2017-01       | fashion_roupa_masculina          |          126.9  |
| 2017-01       | moveis_escritorio                |         1444.35 |
| 2017-01       | seguros_e_servicos               |          183.29 |
| 2017-01       | telefonia_fixa                   |          677.12 |
| 2017-02       |                                  |         9034.1  |
| 2017-02       | audio                            |          163.8  |
| 2017-02       | casa_conforto                    |          414.66 |
| 2017-02       | construcao_ferramentas_seguranca |          379    |
| 2017-02       | fashion_roupa_masculina          |          642.85 |
| 2017-02       | moveis_escritorio                |        11806    |
| 2017-02       | telefonia_fixa                   |          759.23 |
| 2017-03       |                                  |         6714.17 |
| 2017-03       | audio                            |         1134.77 |
| 2017-03       | casa_conforto                    |         2123.4  |
| 2017-03       | casa_conforto_2                  |          342.88 |
| 2017-03       | construcao_ferramentas_seguranca |          380    |
| 2017-03       | fashion_roupa_masculina          |          580.8  |
| 2017-03       | moveis_escritorio                |        11783.5  |
| 2017-03       | telefonia_fixa                   |         1592.35 |
| 2017-04       |                                  |         8693.09 |
| 2017-04       | audio                            |          942.2  |
| 2017-04       | casa_conforto                    |         2411.1  |
| 2017-04       | casa_conforto_2                  |           12.9  |
| 2017-04       | fashion_roupa_masculina          |          416.69 |
| 2017-04       | la_cuisine                       |           24    |
| 2017-04       | moveis_colchao_e_estofado        |          109.4  |
| 2017-04       | moveis_escritorio                |         9411.3  |
| 2017-04       | pc_gamer                         |          259.98 |
| 2017-04       | telefonia_fixa                   |         2998.33 |
| 2017-05       |                                  |        10961.9  |
| 2017-05       | audio                            |         2081.18 |
| 2017-05       | casa_conforto                    |         5293.87 |
| 2017-05       | fashion_roupa_masculina          |          608.9  |
| 2017-05       | la_cuisine                       |          193    |
| 2017-05       | moveis_colchao_e_estofado        |          595.89 |
| 2017-05       | moveis_escritorio                |         8403.43 |
| 2017-05       | pc_gamer                         |          129.99 |
| 2017-05       | telefonia_fixa                   |          462.33 |
| 2017-06       |                                  |         5527.79 |
| 2017-06       | audio                            |         1671    |
| 2017-06       | casa_conforto                    |         3056.19 |
| 2017-06       | casa_conforto_2                  |           12.9  |
| 2017-06       | fashion_roupa_masculina          |          569.69 |
| 2017-06       | moveis_escritorio                |         7279.13 |
| 2017-06       | pc_gamer                         |          129.99 |
| 2017-06       | telefonia_fixa                   |          933.67 |
| 2017-07       |                                  |         3124.66 |
| 2017-07       | artigos_de_festas                |           69.9  |
| 2017-07       | audio                            |          771.69 |
| 2017-07       | casa_conforto                    |         4563.85 |
| 2017-07       | casa_conforto_2                  |           12.9  |
| 2017-07       | fashion_roupa_masculina          |          719.99 |
| 2017-07       | la_cuisine                       |          469.99 |
| 2017-07       | moveis_colchao_e_estofado        |          109.4  |
| 2017-07       | moveis_escritorio                |        16892.6  |
| 2017-07       | telefonia_fixa                   |          218.73 |
| 2017-08       |                                  |         5243.4  |
| 2017-08       | audio                            |         1050.7  |
| 2017-08       | casa_conforto                    |         4000.19 |
| 2017-08       | casa_conforto_2                  |           38.7  |
| 2017-08       | construcao_ferramentas_seguranca |            8.9  |
| 2017-08       | fashion_roupa_masculina          |         1205.05 |
| 2017-08       | la_cuisine                       |          313    |
| 2017-08       | moveis_colchao_e_estofado        |          509.39 |
| 2017-08       | moveis_escritorio                |         9877.36 |
| 2017-08       | telefonia_fixa                   |         3343.54 |
| 2017-09       |                                  |         5904.14 |
| 2017-09       | artigos_de_festas                |          269.8  |
| 2017-09       | audio                            |         1042.59 |
| 2017-09       | casa_conforto                    |         4592.34 |
| 2017-09       | casa_conforto_2                  |           12.9  |
| 2017-09       | construcao_ferramentas_seguranca |         1124.54 |
| 2017-09       | fashion_roupa_masculina          |         1177.5  |
| 2017-09       | moveis_colchao_e_estofado        |           86.5  |
| 2017-09       | moveis_escritorio                |         9208.65 |
| 2017-09       | seguros_e_servicos               |          100    |
| 2017-09       | telefonia_fixa                   |        13692.9  |
| 2017-10       |                                  |        18800.5  |
| 2017-10       | artigos_de_festas                |          137.8  |
| 2017-10       | audio                            |         2348.48 |
| 2017-10       | casa_conforto                    |         3619.8  |
| 2017-10       | casa_conforto_2                  |           64.5  |
| 2017-10       | construcao_ferramentas_seguranca |         1668.85 |
| 2017-10       | fashion_roupa_masculina          |         1041.8  |
| 2017-10       | la_cuisine                       |          137    |
| 2017-10       | moveis_escritorio                |        17890.8  |
| 2017-10       | telefonia_fixa                   |         3751.88 |
| 2017-11       |                                  |        14116    |
| 2017-11       | artigos_de_festas                |          870.69 |
| 2017-11       | audio                            |         3677.23 |
| 2017-11       | casa_conforto                    |         5355.39 |
| 2017-11       | casa_conforto_2                  |           38.7  |

In [ ]:
# 예시 질문
question = "리뷰 점수 3 미만 비율이 20% 이상인 카테고리의 월별 매출 추이"

init = SQLState(
    question=question,
    schema_text=schema_text,
    schema_name=schema_name,
    schema_prefix=schema_prefix,
)

raw_result = app.invoke(init)
state = to_state(raw_result, init)

print("\n🪵 선택된 테이블:", state.selected_tables)
print("🪵 생성된 sub-questions:", state.sub_questions)
print("🪵 생성된 sub-sqls:", state.sub_sqls)
print("🪵 생성된 SQL:", state.sql)
print("🪵 에러 메시지:", state.error)

from IPython.display import Markdown, display
display(Markdown("### ✅ 결과 미리보기\n" + (state.result_preview or "_(출력 없음)_")))

🔎 [selector] raw: {
  "tables": [
    "order_reviews",
    "order_items",
    "products",
    "orders",
    "product_category_name_translations"
  ],
  "rationale": "To calculate the ratio of review scores below 3, `order_reviews` (for `review_score` and `order_id`) is needed. To link reviews to products and categories, `order_items` (for `order_id` and `product_id`) and `products` (for `product_id` and `product_category_name`) are required. To analyze monthly sales trends, `order_items` (for `price` and `freight_value`) and `orders` (for `order_id` and `order_purchase_timestamp`) are necessary. Finally, `produ
🔎 [decomposer] raw: {
  "sub_questions": [
    "리뷰 점수 3 미만 비율이 20% 이상인 카테고리는 무엇인가?",
    "해당 카테고리들의 월별 매출액은 어떻게 변화하는가?"
  ],
  "sub_sqls": [
    "SELECT p.product_category_name, COUNT(CASE WHEN orv.review_score < 3 THEN 1 END) AS low_score_reviews, COUNT(orv.review_score) AS total_reviews FROM ecommerce.orders AS o JOIN ecommerce.order_reviews AS orv ON o.order_id = orv.order_id

### ✅ 결과 미리보기
이 데이터는 리뷰 점수 3점 미만 비율이 20% 이상인 카테고리들을 식별하여, 해당 카테고리들의 월별 매출 추이를 보여줍니다. 각 월별로 카테고리별 총 매출액을 확인할 수 있습니다.

| sales_month | product_category_name_english | monthly_sales_value |
|:------------|:------------------------------|--------------------:|

| sales_month   | product_category_name_english     |   monthly_sales_value |
|:--------------|:----------------------------------|----------------------:|
| 2016-10       | audio                             |                156.99 |
| 2017-02       | audio                             |                163.8  |
| 2017-03       | audio                             |               1134.77 |
| 2017-04       | audio                             |                942.2  |
| 2017-05       | audio                             |               2081.18 |
| 2017-06       | audio                             |               1671    |
| 2017-07       | audio                             |                771.69 |
| 2017-08       | audio                             |               1050.7  |
| 2017-09       | audio                             |               1042.59 |
| 2017-10       | audio                             |               2348.48 |
| 2017-11       | audio                             |               3677.23 |
| 2017-12       | audio                             |               2686.93 |
| 2018-01       | audio                             |               3820.28 |
| 2018-02       | audio                             |               2828.49 |
| 2018-03       | audio                             |              10606.7  |
| 2018-04       | audio                             |               7045.07 |
| 2018-05       | audio                             |               2697.68 |
| 2018-06       | audio                             |               1830.69 |
| 2018-07       | audio                             |               2062.64 |
| 2018-08       | audio                             |               2069.4  |
| 2017-02       | construction_tools_safety         |                379    |
| 2017-03       | construction_tools_safety         |                380    |
| 2017-08       | construction_tools_safety         |                  8.9  |
| 2017-09       | construction_tools_safety         |               1124.54 |
| 2017-10       | construction_tools_safety         |               1668.85 |
| 2017-11       | construction_tools_safety         |               3318.59 |
| 2017-12       | construction_tools_safety         |               1384.92 |
| 2018-01       | construction_tools_safety         |               2228.89 |
| 2018-02       | construction_tools_safety         |               5965.75 |
| 2018-03       | construction_tools_safety         |               5374.87 |
| 2018-04       | construction_tools_safety         |               2887.26 |
| 2018-05       | construction_tools_safety         |               7298.51 |
| 2018-06       | construction_tools_safety         |               2565.37 |
| 2018-07       | construction_tools_safety         |               1888.95 |
| 2018-08       | construction_tools_safety         |               4070.12 |
| 2016-10       | diapers_and_hygiene               |                134.9  |
| 2017-12       | diapers_and_hygiene               |                 76    |
| 2018-01       | diapers_and_hygiene               |                125    |
| 2018-02       | diapers_and_hygiene               |                100    |
| 2018-03       | diapers_and_hygiene               |                163    |
| 2018-04       | diapers_and_hygiene               |                149    |
| 2018-05       | diapers_and_hygiene               |                117    |
| 2018-06       | diapers_and_hygiene               |                321    |
| 2018-07       | diapers_and_hygiene               |                149.8  |
| 2018-08       | diapers_and_hygiene               |                231.89 |
| 2016-10       | fashion_male_clothing             |                 24.9  |
| 2017-01       | fashion_male_clothing             |                126.9  |
| 2017-02       | fashion_male_clothing             |                642.85 |
| 2017-03       | fashion_male_clothing             |                580.8  |
| 2017-04       | fashion_male_clothing             |                416.69 |
| 2017-05       | fashion_male_clothing             |                608.9  |
| 2017-06       | fashion_male_clothing             |                569.69 |
| 2017-07       | fashion_male_clothing             |                719.99 |
| 2017-08       | fashion_male_clothing             |               1205.05 |
| 2017-09       | fashion_male_clothing             |               1177.5  |
| 2017-10       | fashion_male_clothing             |               1041.8  |
| 2017-11       | fashion_male_clothing             |                929.99 |
| 2017-12       | fashion_male_clothing             |                450.89 |
| 2018-01       | fashion_male_clothing             |                550.6  |
| 2018-02       | fashion_male_clothing             |                388.09 |
| 2018-04       | fashion_male_clothing             |                195    |
| 2018-05       | fashion_male_clothing             |                129.99 |
| 2018-06       | fashion_male_clothing             |                447.8  |
| 2018-07       | fashion_male_clothing             |                208.7  |
| 2018-08       | fashion_male_clothing             |                381.69 |
| 2016-10       | fixed_telephony                   |                704.88 |
| 2017-01       | fixed_telephony                   |                677.12 |
| 2017-02       | fixed_telephony                   |                759.23 |
| 2017-03       | fixed_telephony                   |               1592.35 |
| 2017-04       | fixed_telephony                   |               2998.33 |
| 2017-05       | fixed_telephony                   |                462.33 |
| 2017-06       | fixed_telephony                   |                933.67 |
| 2017-07       | fixed_telephony                   |                218.73 |
| 2017-08       | fixed_telephony                   |               3343.54 |
| 2017-09       | fixed_telephony                   |              13692.9  |
| 2017-10       | fixed_telephony                   |               3751.88 |
| 2017-11       | fixed_telephony                   |                943.98 |
| 2017-12       | fixed_telephony                   |                998.77 |
| 2018-01       | fixed_telephony                   |               2444.1  |
| 2018-02       | fixed_telephony                   |               6138.67 |
| 2018-03       | fixed_telephony                   |               1930.59 |
| 2018-04       | fixed_telephony                   |               1921.94 |
| 2018-05       | fixed_telephony                   |               1946.1  |
| 2018-06       | fixed_telephony                   |               2080.32 |
| 2018-07       | fixed_telephony                   |               9319.39 |
| 2018-08       | fixed_telephony                   |               2724.17 |
| 2017-04       | furniture_mattress_and_upholstery |                109.4  |
| 2017-05       | furniture_mattress_and_upholstery |                595.89 |
| 2017-07       | furniture_mattress_and_upholstery |                109.4  |
| 2017-08       | furniture_mattress_and_upholstery |                509.39 |
| 2017-09       | furniture_mattress_and_upholstery |                 86.5  |
| 2017-11       | furniture_mattress_and_upholstery |                334    |
| 2017-12       | furniture_mattress_and_upholstery |                803.7  |
| 2018-01       | furniture_mattress_and_upholstery |                860.5  |
| 2018-02       | furniture_mattress_and_upholstery |                389.4  |
| 2018-03       | furniture_mattress_and_upholstery |                440    |
| 2018-07       | furniture_mattress_and_upholstery |                129.9  |
| 2017-01       | home_comfort_2                    |                 72.8  |
| 2017-03       | home_comfort_2                    |                342.88 |
| 2017-04       | home_comfort_2                    |                 12.9  |

In [ ]:
question = "프로모션 사용/미사용 주문의 주문수, 총매출, 객단가 비교"

init = SQLState(
    question=question,
    schema_text=schema_text,
    schema_name=schema_name,
    schema_prefix=schema_prefix,
)

raw_result = app.invoke(init)
state = to_state(raw_result, init)

print("\n🪵 선택된 테이블:", state.selected_tables)
print("🪵 생성된 sub-questions:", state.sub_questions)
print("🪵 생성된 sub-sqls:", state.sub_sqls)
print("🪵 생성된 SQL:", state.sql)
print("🪵 에러 메시지:", state.error)

from IPython.display import Markdown, display
display(Markdown("### ✅ 결과 미리보기\n" + (state.result_preview or "_(출력 없음)_")))

🔎 [selector] raw: {
  "tables": [
    "orders",
    "order_payments"
  ],
  "rationale": "To compare order count, total sales, and average order value for promotion used/unused orders, we need the 'orders' table to count orders and the 'order_payments' table to identify orders that used a 'voucher' (promotion) via the 'payment_type' column and to sum 'payment_value' for total sales. The average order value can then be calculated from these two metrics."
}
🔎 [decomposer] raw: {
  "sub_questions": [],
  "sub_sqls": [],
  "final_sql": "WITH OrderTotalValue AS (\n    SELECT\n        oi.order_id,\n        SUM(oi.price + oi.freight_value) AS total_order_value\n    FROM\n        ecommerce.order_items oi\n    GROUP BY\n        oi.order_id\n),\nOrderPromotionStatus AS (\n    SELECT\n        o.order_id,\n        CASE\n            WHEN EXISTS (SELECT 1 FROM ecommerce.order_payments op WHERE op.order_id = o.order_id AND op.payment_type = 'voucher') THEN 'Promotion Used'\n            ELSE 'Promotio

### ✅ 결과 미리보기
프로모션 사용 여부에 따른 주문수, 총매출, 객단가를 비교 분석한 결과입니다. 샘플 데이터에 따르면, 프로모션을 사용하지 않은 주문이 사용한 주문보다 주문수, 총매출, 객단가 모두 더 높게 나타났습니다.

| promotion_status | 주문수 | 총매출 | 객단가 |
|:-----------------|-------:|-------:|-------:|

| promotion_status   |   주문수 |          총매출 |   객단가 |
|:-------------------|---------:|----------------:|---------:|
| Promotion Not Used |    94900 |      1.5347e+07 |  161.717 |
| Promotion Used     |     3766 | 496583          |  131.86  |

In [ ]:
question = "프로모션 기간 주문의 순 매출(할인 차감 후), 객단가(AOV), 리뷰 평균점수를 비프로모션 기간과 비교해줘."

init = SQLState(
    question=question,
    schema_text=schema_text,
    schema_name=schema_name,
    schema_prefix=schema_prefix,
)

raw_result = app.invoke(init)
state = to_state(raw_result, init)

print("\n🪵 선택된 테이블:", state.selected_tables)
print("🪵 생성된 sub-questions:", state.sub_questions)
print("🪵 생성된 sub-sqls:", state.sub_sqls)
print("🪵 생성된 SQL:", state.sql)
print("🪵 에러 메시지:", state.error)

from IPython.display import Markdown, display
display(Markdown("### ✅ 결과 미리보기\n" + (state.result_preview or "_(출력 없음)_")))

🔎 [selector] raw: {
  "tables": [
    "orders",
    "order_payments",
    "order_reviews"
  ],
  "rationale": "To calculate '순 매출(할인 차감 후)' and '객단가(AOV)', the 'order_payments' table is essential for 'payment_value' (total amount paid per order) and the 'orders' table is needed to count orders and group them by 'order_id'. For '리뷰 평균점수', the 'order_reviews' table is required for 'review_score'. The 'orders' table is also crucial for its 'order_purchase_timestamp' to distinguish between '프로모션 기간' and '비프로모션 기간' for comparison."
}
🔎 [decomposer] raw: {
  "sub_questions": [
    "프로모션 기간을 정의할 수 있는 명시적인 컬럼이 스키마에 존재하지 않아, 'voucher' 결제 수단이 포함된 주문을 '프로모션 기간 주문'으로 가정하고 분석을 진행합니다. 이 가정에 대한 확인이 필요합니다.",
    "순 매출은 'delivered' 상태의 주문에 대해 각 주문의 총 결제 금액(ecommerce.order_payments.payment_value 합계)으로 정의합니다.",
    "객단가(AOV)는 순 매출을 'delivered' 상태의 고유 주문 건수로 나눈 값으로 계산합니다."
  ],
  "sub_sqls": [
    "SELECT DISTINCT ecommerce.order_payments.order_id FROM ecommerce.order_payments WHERE ecommerce.order_paymen

### ✅ 결과 미리보기
프로모션 기간과 비프로모션 기간의 주문 데이터를 비교 분석한 결과, 프로모션 기간의 순 매출과 객단가(AOV)는 비프로모션 기간보다 낮게 나타났습니다. 반면, 리뷰 평균 점수는 두 기간 모두 4점대 초반으로 큰 차이를 보이지 않았습니다.

| period_type | net_revenue | average_order_value | average_review_score |
|:------------|------------:|--------------------:|---------------------:|

| period_type                 |      net_revenue |   average_order_value |   average_review_score |
|:----------------------------|-----------------:|----------------------:|-----------------------:|
| Non-Promotion Period Orders |      1.50073e+07 |               161.72  |                4.15746 |
| Promotion Period Orders     | 485023           |               131.835 |                4.12495 |

In [ ]:
question = "카테고리별로 프로모션 노출 → 장바구니 → 구매 전환율을 계산하고, 전환율 상위 5개와 하위 5개 카테고리를 보여줘."

init = SQLState(
    question=question,
    schema_text=schema_text,
    schema_name=schema_name,
    schema_prefix=schema_prefix,
)

raw_result = app.invoke(init)
state = to_state(raw_result, init)

print("\n🪵 선택된 테이블:", state.selected_tables)
print("🪵 생성된 sub-questions:", state.sub_questions)
print("🪵 생성된 sub-sqls:", state.sub_sqls)
print("🪵 생성된 SQL:", state.sql)
print("🪵 에러 메시지:", state.error)

from IPython.display import Markdown, display
display(Markdown("### ✅ 결과 미리보기\n" + (state.result_preview or "_(출력 없음)_")))

🔎 [selector] raw: {
  "tables": [
    "ecommerce.products",
    "ecommerce.order_items",
    "ecommerce.orders"
  ],
  "rationale": "To calculate conversion rates by category, we need to identify products and their categories, and then link them to purchase events. The 'ecommerce.products' table provides product IDs and their categories. The 'ecommerce.order_items' table links product IDs to specific order IDs. The 'ecommerce.orders' table contains the order status and timestamps, which are necessary to confirm a completed purchase. However, the schema does not contain data for 'promotion exposure' or 'add to c
🔎 [decomposer] raw: {
  "sub_questions": [
    "\"프로모션 노출\" (Promotion Exposure) 및 \"장바구니\" (Add to Cart) 정의: 스키마에 직접적인 프로모션 노출 또는 장바구니 이벤트 로그가 없으므로, 본 쿼리에서는 ecommerce.order_items 테이블에 기록된 각 상품 항목을 \"장바구니에 담긴\" 또는 \"주문 시도된\" 상품으로 간주합니다. 따라서 \"프로모션 노출\" 단계는 \"장바구니\" 단계와 동일하게 처리됩니다.",
    "\"구매\" (Purchase) 정의: ecommerce.orders 테이블의 order_approved_at 컬럼이 NULL이 아닌 경우 해당 주문이 승인된 것으로

### ✅ 결과 미리보기
각 카테고리별로 장바구니 담기 대비 구매 전환율을 계산했습니다. 이 분석을 통해 전환율이 가장 높은 상위 5개 카테고리와 가장 낮은 하위 5개 카테고리를 확인할 수 있습니다.

| 구분 | 카테고리 | 전환율 |
|:---|:---|---:|

| rank_type   | product_category_name_english   |   conversion_rate |   sort_order_group |   sort_rate_desc |   sort_rate_asc |
|:------------|:--------------------------------|------------------:|-------------------:|-----------------:|----------------:|
| Top 5       | air_conditioning                |          1        |                  1 |                1 |      nan        |
| Top 5       | art                             |          1        |                  1 |                1 |      nan        |
| Top 5       | arts_and_craftmanship           |          1        |                  1 |                1 |      nan        |
| Top 5       | audio                           |          1        |                  1 |                1 |      nan        |
| Top 5       | agro_industry_and_commerce      |          1        |                  1 |                1 |      nan        |
| Bottom 5    | construction_tools_safety       |          0.994845 |                  2 |              nan |        0.994845 |
| Bottom 5    | small_appliances                |          0.998527 |                  2 |              nan |        0.998527 |
| Bottom 5    | cool_stuff                      |          0.99921  |                  2 |              nan |        0.99921  |
| Bottom 5    | telephony                       |          0.99956  |                  2 |              nan |        0.99956  |
| Bottom 5    | furniture_decor                 |          0.99964  |                  2 |              nan |        0.99964  |

In [ ]:
question = "카테고리별 가격 변동(분위/버킷)에 따른 수요량 변화를 집계하고, 프로모션 여부를 상호작용 항으로 넣어 민감도가 높은 카테고리 Top 5를 찾아줘."

init = SQLState(
    question=question,
    schema_text=schema_text,
    schema_name=schema_name,
    schema_prefix=schema_prefix,
)

raw_result = app.invoke(init)
state = to_state(raw_result, init)

print("\n🪵 선택된 테이블:", state.selected_tables)
print("🪵 생성된 sub-questions:", state.sub_questions)
print("🪵 생성된 sub-sqls:", state.sub_sqls)
print("🪵 생성된 SQL:", state.sql)
print("🪵 에러 메시지:", state.error)

from IPython.display import Markdown, display
display(Markdown("### ✅ 결과 미리보기\n" + (state.result_preview or "_(출력 없음)_")))

🔎 [selector] raw: {
  "tables": [
    "ecommerce.products",
    "ecommerce.order_items",
    "ecommerce.order_payments"
  ],
  "rationale": "To analyze '카테고리별 가격 변동에 따른 수요량 변화' and '프로모션 여부' as an interaction term, the following tables are essential: `ecommerce.products` for `product_category_name` (category), `ecommerce.order_items` for `price` (price variation) and to quantify demand (by counting items), and `ecommerce.order_payments` to infer '프로모션 여부' (promotion status) by examining `payment_type` (e.g., 'voucher'). These tables can be joined via `products.product_id` to `order_items.product_id` and `order_
🔎 [decomposer] raw: {
  "sub_questions": [
    "How can price variation be categorized into buckets, and how is demand quantified for each category and price bucket?",
    "What is the definition of 'promotion status' based on the available schema, and how is it applied?",
    "How is 'sensitivity' defined and calculated, and how is the 'promotion status' incorporated as an inte

### ✅ 결과 미리보기
카테고리별 가격 분위(quintile)와 프로모션 여부가 수요량에 미치는 영향을 분석했습니다. 이를 통해 프로모션 여부에 따라 가격 민감도가 가장 높은 상위 5개 카테고리를 식별했습니다.

| product_category_name_english | sensitivity_promoted | sensitivity_non_promoted | interaction_sensitivity_score |
|:------------------------------|---------------------:|-------------------------:|------------------------------:|

| product_category_name_english   |   sensitivity_promoted |   sensitivity_non_promoted |   interaction_sensitivity_score |
|:--------------------------------|-----------------------:|---------------------------:|--------------------------------:|
| sports_leisure                  |                     47 |                        -46 |                              93 |
| bed_bath_table                  |                     42 |                        -42 |                              84 |
| health_beauty                   |                     40 |                        -40 |                              80 |
| computers_accessories           |                     36 |                        -35 |                              71 |
| furniture_decor                 |                     31 |                        -30 |                              61 |

In [ ]:
question = "어떤 플친 페이지가 구매 전환이 제일 좋았어?"

init = SQLState(
    question=question,
    schema_text=schema_text,
    schema_name=schema_name,
    schema_prefix=schema_prefix,
)

raw_result = app.invoke(init)
state = to_state(raw_result, init)

print("\n🪵 선택된 테이블:", state.selected_tables)
print("🪵 생성된 sub-questions:", state.sub_questions)
print("🪵 생성된 sub-sqls:", state.sub_sqls)
print("🪵 생성된 SQL:", state.sql)
print("🪵 에러 메시지:", state.error)

from IPython.display import Markdown, display
display(Markdown("### ✅ 결과 미리보기\n" + (state.result_preview or "_(출력 없음)_")))

🔎 [selector] raw: {
  "tables": [],
  "rationale": "The schema does not contain any information about '플친 페이지' (KakaoTalk Plus Friend page) or any other marketing channels or sources that would allow tracking purchase conversion by channel. Therefore, the question cannot be answered with the provided tables."
}
🔎 [decomposer] raw: {
  "sub_questions": [
    "What does \"플친 페이지\" (friend page) refer to in the given e-commerce schema?",
    "How is \"구매 전환\" (purchase conversion) defined and calculated in this context?",
    "How can we link orders to these \"pages\" and aggregate the necessary metrics to find the best performing one?"
  ],
  "sub_sqls": [
    "SELECT DISTINCT ecommerce.product_category_name_translations.product_category_name_english FROM ecommerce.product_category_name_translations;",
    "SELECT ecommerce.orders.order_id, ecommerce.orders.order_status FROM ecommerce.orders LIMIT 5;",
    "SELECT tpct.product_category_name_english, COUNT(DISTINCT to2.order_id) AS total_

### ✅ 결과 미리보기
구매 전환율이 가장 높은 제품 카테고리를 분석한 결과, 'arts_and_craftmanship' 카테고리가 100%의 전환율로 가장 높은 성과를 보였습니다. 이는 해당 카테고리의 모든 주문이 성공적으로 배송되었음을 의미합니다.

| product_category_name_english | conversion_rate |
|:--------------------------------|------------------:|

| product_category_name_english   |   conversion_rate |
|:--------------------------------|------------------:|
| arts_and_craftmanship           |               100 |

In [ ]:
question = "연령대(age_group)×카테고리 조합별 구매율/객단가/반품률을 집계하고, 구매율 상위 10개 조합을 보여줘. "

init = SQLState(
    question=question,
    schema_text=schema_text,
    schema_name=schema_name,
    schema_prefix=schema_prefix,
)

raw_result = app.invoke(init)
state = to_state(raw_result, init)

print("\n🪵 선택된 테이블:", state.selected_tables)
print("🪵 생성된 sub-questions:", state.sub_questions)
print("🪵 생성된 sub-sqls:", state.sub_sqls)
print("🪵 생성된 SQL:", state.sql)
print("🪵 에러 메시지:", state.error)

from IPython.display import Markdown, display
display(Markdown("### ✅ 결과 미리보기\n" + (state.result_preview or "_(출력 없음)_")))

🔎 [selector] raw: {
  "tables": [
    "customers",
    "orders",
    "order_items",
    "products",
    "order_payments"
  ],
  "rationale": "To calculate purchase rate, average order value (객단가), and return rate by age group and category, the following tables are required: 'customers' for 'age_group' and customer information; 'products' for 'product_category_name'; 'order_items' to link orders to products; 'orders' to get 'order_id', 'customer_id', and 'order_status' (for purchase/return status); and 'order_payments' for 'payment_value' to calculate the total order value for '객단가'."
}
🔎 [decomposer] raw: {
  "sub_questions": [
    "Calculate the total number of unique customers for each age group to be used as the denominator for purchase rate.",
    "Aggregate key metrics (total orders, successful orders, canceled orders, total purchase value, and unique customers who purchased a specific category) for each combination of age group and product category.",
    "Join the aggregated pur

### ✅ 결과 미리보기
이 데이터는 연령대와 카테고리 조합별 구매율, 객단가, 반품률을 분석하여 구매율이 가장 높은 상위 10개 조합을 보여줍니다. 20대와 30대에서는 '침구/욕실용품(bed_bath_table)' 카테고리의 구매율이 특히 높게 나타났으며, '건강/미용(health_beauty)' 카테고리도 여러 연령대에서 상위권을 차지했습니다. 각 조합별로 객단가와 반품률은 큰 차이를 보였습니다.

| age_group | category | purchase_rate | average_order_value | return_rate |
|:----------|:---------|--------------:|--------------------:|------------:|

| age_group   | category       |   purchase_rate |   average_order_value |   return_rate |
|:------------|:---------------|----------------:|----------------------:|--------------:|
| 20s         | bed_bath_table |        10.6765  |               91.874  |     0.331004  |
| 30s         | bed_bath_table |        10.5469  |              149.179  |     0.036324  |
| 10s         | telephony      |         9.52063 |               42.4084 |     0.0930233 |
| 20s         | health_beauty  |         9.27737 |               87.9495 |     0.381518  |
| 30s         | health_beauty  |         9.23698 |              185.858  |     0.417885  |
| 40s         | health_beauty  |         9.17533 |              320.914  |     0.328484  |
| 40s         | watches_gifts  |         8.9665  |              352.018  |     0.193892  |
| 40s         | bed_bath_table |         8.93095 |              218.965  |     0.139405  |
| 10s         | health_beauty  |         8.64906 |               54.7928 |     0.512033  |
| 30s         | sports_leisure |         8.6196  |              167.553  |     0.88968   |

In [ ]:
question = "전체 기간 기준 매출 Top 5 카테고리의 총매출을 알려줘.(a) 각각의 총매출을 보여주고, (b) Top 5 합계도 알려줘."

init = SQLState(
    question=question,
    schema_text=schema_text,
    schema_name=schema_name,
    schema_prefix=schema_prefix,
)

raw_result = app.invoke(init)
state = to_state(raw_result, init)

print("\n🪵 선택된 테이블:", state.selected_tables)
print("🪵 생성된 sub-questions:", state.sub_questions)
print("🪵 생성된 sub-sqls:", state.sub_sqls)
print("🪵 생성된 SQL:", state.sql)
print("🪵 에러 메시지:", state.error)

from IPython.display import Markdown, display
display(Markdown("### ✅ 결과 미리보기\n" + (state.result_preview or "_(출력 없음)_")))

🔎 [selector] raw: {
  "tables": [
    "ecommerce.order_items",
    "ecommerce.products",
    "ecommerce.product_category_name_translations"
  ],
  "rationale": "To calculate the total sales per category, we need 'ecommerce.order_items' for the 'price' and 'freight_value' of each item, and 'ecommerce.products' to link 'product_id' to 'product_category_name'. 'ecommerce.product_category_name_translations' is included to provide the English names for the product categories, which is generally preferred for analysis and presentation."
}
🔎 [decomposer] raw: {
  "sub_questions": [
    "Calculate the total sales for each product category.",
    "Identify the top 5 product categories based on their total sales.",
    "Display the total sales for each of these top 5 categories.",
    "Calculate and display the sum of total sales for these top 5 categories."
  ],
  "sub_sqls": [
    "WITH category_total_sales AS (SELECT t.product_category_name_english, SUM(oi.price) AS total_sales FROM ecommerce

### ✅ 결과 미리보기
전체 기간 기준 매출 상위 5개 카테고리의 총매출을 분석했습니다. 각 카테고리별 총매출과 함께, 이들 상위 5개 카테고리의 총매출 합계를 확인할 수 있습니다.

| 카테고리명 | 총매출 |
|:---|---:|

| category_name         |      total_sales |   sort_order_key |
|:----------------------|-----------------:|-----------------:|
| health_beauty         |      1.25868e+06 |                0 |
| watches_gifts         |      1.20501e+06 |                0 |
| bed_bath_table        |      1.03699e+06 |                0 |
| sports_leisure        | 988049           |                0 |
| computers_accessories | 911954           |                0 |
| Total Top 5 Sales     |      5.40068e+06 |                1 |

In [ ]:
question = "전체 기간 기준 매출 Top 5 카테고리의 총매출을 알려줘.(a) 각각의 총매출을 보여주고, (b) Top 5 합계도 알려줘."

init = SQLState(
    question=question,
    schema_text=schema_text,
    schema_name=schema_name,
    schema_prefix=schema_prefix,
)

raw_result = app.invoke(init)
state = to_state(raw_result, init)

print("\n🪵 선택된 테이블:", state.selected_tables)
print("🪵 생성된 sub-questions:", state.sub_questions)
print("🪵 생성된 sub-sqls:", state.sub_sqls)
print("🪵 생성된 SQL:", state.sql)
print("🪵 에러 메시지:", state.error)

from IPython.display import Markdown, display
display(Markdown("### ✅ 결과 미리보기\n" + (state.result_preview or "_(출력 없음)_")))

🔎 [selector] raw: {
  "tables": [
    "order_items",
    "products"
  ],
  "rationale": "To determine the total sales per product category, we need the `order_items` table to access the `price`, `quantity`, and `freight_value` for each ordered product, along with the `product_id`. The `products` table is required to link the `product_id` to its corresponding `product_category`. By joining these two tables, we can calculate the total sales for each category and identify the top 5."
}
🔎 [decomposer] raw: {
  "sub_questions": [
    "Calculate the total revenue for each product category.",
    "Identify the top 5 product categories based on their total revenue.",
    "Display the total revenue for each of the top 5 categories and also the sum of their total revenues."
  ],
  "sub_sqls": [
    "WITH category_revenue AS (SELECT p.product_category, SUM(oi.price * oi.quantity) AS total_revenue FROM ecommerce.order_items AS oi JOIN ecommerce.products AS p ON oi.product_id = p.product_id WHERE p

### ✅ 결과 미리보기
전체 기간 동안의 매출을 기준으로 상위 5개 카테고리의 총매출액을 분석했습니다. 각 카테고리별 총매출액과 함께, 이들 상위 5개 카테고리의 총매출 합계도 확인하실 수 있습니다.

| product_category | total_revenue |
|:-----------------|--------------:|

| product_category      |    total_revenue |
|:----------------------|-----------------:|
| health_beauty         |      1.25868e+06 |
| watches_gifts         |      1.20501e+06 |
| bed_bath_table        |      1.03699e+06 |
| sports_leisure        | 988049           |
| computers_accessories | 911954           |
| Top 5 Total           |      5.40068e+06 |

In [ ]:
question = "2024년 1월 각 요일별(day_of_week)로 마케팅 캠페인 메시지가 발송된 총 횟수와 해당 캠페인을 통한 평균 구매전환율(conversion_rate)을 보여주세요. (캠페인 유형이 'promotion'인 경우만 포함)"

init = SQLState(
    question=question,
    schema_text=schema_text,
    schema_name=schema_name,
    schema_prefix=schema_prefix,
)

raw_result = app.invoke(init)
state = to_state(raw_result, init)

print("\n🪵 선택된 테이블:", state.selected_tables)
print("🪵 생성된 sub-questions:", state.sub_questions)
print("🪵 생성된 sub-sqls:", state.sub_sqls)
print("🪵 생성된 SQL:", state.sql)
print("🪵 에러 메시지:", state.error)

from IPython.display import Markdown, display
display(Markdown("### ✅ 결과 미리보기\n" + (state.result_preview or "_(출력 없음)_")))

🔎 [selector] raw: {
  "tables": [
    "marketing_campaign"
  ],
  "rationale": "The question asks for the total count of marketing campaign messages sent and the average conversion rate per day of the week for January 2024, specifically for 'promotion' campaign types. The 'marketing_campaign' table contains all the necessary columns: 'send_timestamp' for filtering by date, 'day_of_week' for grouping, 'campaign_type' for filtering by 'promotion', and 'conversion_rate' for calculating the average. No joins are required."
}
🔎 [decomposer] raw: {
  "sub_questions": [
    "What table contains information about marketing campaigns, including send time, day of week, campaign type, and conversion rate?",
    "How can I filter the marketing campaign data to include only messages sent in January 2024?",
    "How can I filter the marketing campaign data to include only campaigns of type 'promotion'?",
    "How do I calculate the total count of marketing messages sent for each day of the week?",
 

### ✅ 결과 미리보기
2024년 1월 한 달간 'promotion' 유형의 마케팅 캠페인 메시지 발송 현황을 요일별로 분석했습니다. 각 요일별 총 발송 횟수와 해당 캠페인의 평균 구매전환율을 확인할 수 있습니다.

| 요일 | 총 발송 횟수 | 평균 구매전환율 |
|:---|:---|:---|

| day_of_week   |   total_messages_sent |   average_conversion_rate |
|:--------------|----------------------:|--------------------------:|
| Tuesday       |                     1 |                     1.944 |
| Wednesday     |                     1 |                     2.334 |
| Thursday      |                     1 |                     1.161 |

In [ ]:
question = "월별(order_purchase_timestamp 기준)로 가장 많은 리뷰 점수(review_score) 평균을 기록한 상품 카테고리는 무엇이며, 해당 카테고리의 월별 평균 리뷰 점수와 해당 월의 전체 평균 리뷰 점수를 함께 보여주세요."
init = SQLState(
    question=question,
    schema_text=schema_text,
    schema_name=schema_name,
    schema_prefix=schema_prefix,
)

raw_result = app.invoke(init)
state = to_state(raw_result, init)

print("\n🪵 선택된 테이블:", state.selected_tables)
print("🪵 생성된 sub-questions:", state.sub_questions)
print("🪵 생성된 sub-sqls:", state.sub_sqls)
print("🪵 생성된 SQL:", state.sql)
print("🪵 에러 메시지:", state.error)

from IPython.display import Markdown, display
display(Markdown("### ✅ 결과 미리보기\n" + (state.result_preview or "_(출력 없음)_")))

🔎 [selector] raw: {
  "tables": [
    "orders",
    "order_reviews",
    "order_items",
    "products"
  ],
  "rationale": "To answer the question, we need the order purchase timestamp (`orders.order_purchase_timestamp`) to group by month, the review score (`order_reviews.review_score`), and the product category (`products.product_category`). The `order_items` table is necessary to link `orders` and `order_reviews` (via `order_id`) to `products` (via `product_id`). The `product_categories` table is not strictly required as `products.product_category` already contains the category name needed for grouping and di
🔎 [decomposer] raw: {
  "sub_questions": [
    "월별로 각 상품 카테고리별 평균 리뷰 점수를 계산합니다.",
    "월별로 전체 상품의 평균 리뷰 점수를 계산합니다.",
    "각 월별로 가장 높은 평균 리뷰 점수를 기록한 상품 카테고리를 식별합니다.",
    "최종적으로 월, 해당 월의 최고 리뷰 점수 카테고리, 해당 카테고리의 월별 평균 리뷰 점수, 그리고 해당 월의 전체 평균 리뷰 점수를 함께 조회합니다."
  ],
  "sub_sqls": [
    "WITH MonthlyCategoryAvg AS (SELECT TO_CHAR(o.order_purchase_timestamp, 'YYYY-MM') AS sales_month, 

### ✅ 결과 미리보기
각 월별로 가장 높은 평균 리뷰 점수를 기록한 상품 카테고리와 해당 카테고리의 월별 평균 리뷰 점수를 보여줍니다. 또한, 해당 월의 전체 상품 평균 리뷰 점수와 비교하여 특정 카테고리의 성과를 확인할 수 있습니다. 결과에 따르면, 많은 월에서 특정 카테고리가 5점 만점의 평균 리뷰 점수를 달성하며 전체 평균을 상회하는 경향을 보였습니다.

| 월 | 가장_높은_리뷰_점수_카테고리 | 해당_카테고리_월별_평균_리뷰_점수 | 해당_월_전체_평균_리뷰_점수 |
|:--------|:-------------------------------|------------------------------------:|------------------------------:|

| 월      | 가장_높은_리뷰_점수_카테고리   |   해당_카테고리_월별_평균_리뷰_점수 |   해당_월_전체_평균_리뷰_점수 |
|:--------|:-------------------------------|------------------------------------:|------------------------------:|
| 2023-06 | telephony                      |                                   1 |                       1       |
| 2023-07 | audio                          |                                   5 |                       3.64706 |
| 2023-10 | auto                           |                                   5 |                       4.13462 |
| 2023-11 | costruction_tools_garden       |                                   5 |                       4.11749 |
| 2023-12 | computers                      |                                   5 |                       4.02441 |
| 2024-01 | books_technical                |                                   5 |                       4.08397 |
| 2024-02 | agro_industry_and_commerce     |                                   5 |                       4.0632  |
| 2024-03 | art                            |                                   5 |                       4.18209 |
| 2024-04 | arts_and_craftmanship          |                                   5 |                       4.19638 |
| 2024-05 | agro_industry_and_commerce     |                                   5 |                       4.17083 |
| 2024-06 | agro_industry_and_commerce     |                                   5 |                       4.23054 |
| 2024-07 | books_imported                 |                                   5 |                       4.12809 |
| 2024-08 | cds_dvds_musicals              |                                   5 |                       4.12611 |
| 2024-09 | art                            |                                   5 |                       3.88088 |
| 2024-10 | cine_photo                     |                                   5 |                       4.07872 |
| 2024-11 | cine_photo                     |                                   5 |                       3.97907 |
| 2024-12 | dvds_blu_ray                   |                                   5 |                       3.64502 |
| 2025-01 | books_imported                 |                                   5 |                       3.97396 |
| 2025-02 | books_imported                 |                                   5 |                       4.16148 |
| 2025-03 | christmas_supplies             |                                   5 |                       4.15509 |
| 2025-04 | costruction_tools_tools        |                                   5 |                       4.29232 |
| 2025-05 | costruction_tools_tools        |                                   5 |                       4.20989 |
| 2025-06 | dvds_blu_ray                   |                                   5 |                       4.32393 |